# Предобработка данных с продажами автомобилей в Казахстане

Мы имеем агрегированные данные по авторынку — дан датасет `auto_kz_2019.csv` с продажами автомобилей в Казахстане за $2019$ год в период с января по сентябрь.

**Цель:** провести предобработку датасета, чтобы он был пригодным для анализа. 

**Задачи и план действий:**

1. Обзор данных
    1. Обзор колонок
<br>
<br>
2. Предобработка данных
    1. Работа с колонками
        1. Удаление
        2. Переименование
        3. Создание
        4. Замена типов данных
    2. "Олицетворение" данных
    3. Дубликаты и аномалии
    4. Пропущенные значения
    5. Ошибки в классификациях

<br>

## Обзор данных

Сначала импортируем нужные библиотеки и зададим опции.

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np

# отображение неограниченного количества колонок в выводе
pd.set_option('display.max_columns', None)

Прочитаем csv файл.

In [2]:
# чтение файла
df = pd.read_csv('https://raw.githubusercontent.com/SweexFox/portfolio-projects/main/python-projects/7-kz-car-market/raw_auto_kz.csv', sep=';')

Посмотрим первые строки.

In [3]:
df

,Год,Месяц,Компания,Бренд,Модель,Модификация,Год выпуска,Страна-производитель,Вид топлива,"Объём двиг, л,",Коробка передач,Тип привода,Сегмент,Регион,Наименование дилерского центра,Тип клиента,Форма расчета,Количество,"Цена, USD","Продажа, USD",Область,Сегментация 2013,Класс 2013,Сегментация Eng,Локализация производства
0,2019,Май,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.0,28115,28115,г.Алматы,Легковые автомобили,C класс,C,Импорт
1,2019,Август,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,наличный,1.0,"32246,99","32246,99",г.Алматы,Легковые автомобили,C класс,C,Импорт
2,2019,Апрель,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.0,32000,32000,г.Алматы,Легковые автомобили,D класс,D,Импорт
3,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,безналичный,1.0,31929,31929,г.Алматы,Легковые автомобили,D класс,D,Импорт
4,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,наличный,1.0,31929,31929,г.Алматы,Легковые автомобили,D класс,D,Импорт
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39961,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,KAMAZ 58815Z,2019,Республика Казахстан,Дизель,NaN,MT,NaN,Грузовой автомобиль,Актобе,ЗапКазКАМАЗ -Актобе,Юр. Лицо,Безналичная,1.0,"48756,92071","48756,92071",Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,O,Локальное производство
39962,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,KAMAZ 58815Z,2019,Республика Казахстан,Дизель,NaN,MT,NaN,Грузовой автомобиль,Актобе,ЗапКазКАМАЗ -Актобе,Юр. Лицо,Безналичная,1.0,"48756,92071","48756,92071",Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,O,Локальное производство
39963,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,KAMAZ 58815Z,2019,Республика Казахстан,Дизель,NaN,MT,NaN,Грузовой автомобиль,Актобе,ЗапКазКАМАЗ -Актобе,Юр. Лицо,Безналичная,1.0,"48756,92071","48756,92071",Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,O,Локальное производство
39964,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,KAMAZ 58815Z,2019,Республика Казахстан,Дизель,NaN,MT,NaN,Грузовой автомобиль,Актобе,ЗапКазКАМАЗ -Актобе,Юр. Лицо,Безналичная,1.0,"48756,92071","48756,92071",Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики,O,Локальное производство


Что сразу бросается в глаза, так это отсутствие первичных ключей в таблице, данные кажутся обезличенными.

In [4]:
df.query('Модель == "Corolla"')\
  .pivot_table(index=['Модель', 'Объём двиг, л,'], values='Бренд', aggfunc='count')

Бренд
Модель  Объём двиг, л,       
Corolla #Н/Д                2
        1,33                7
        1,6               176
        1.6                14
        2,7               523
        2.7               236
        4                  55
        4.0                76

<br>

Посмотрим сводную информацию о таблице.

In [5]:
# общая информация
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Год                             39966 non-null  int64  
 1   Месяц                           39966 non-null  object 
 2   Компания                        39966 non-null  object 
 3   Бренд                           39966 non-null  object 
 4   Модель                          39966 non-null  object 
 5   Модификация                     36375 non-null  object 
 6   Год выпуска                     39465 non-null  object 
 7   Страна-производитель            39966 non-null  object 
 8   Вид топлива                     36826 non-null  object 
 9   Объём двиг, л,                  35708 non-null  object 
 10  Коробка передач                 36711 non-null  object 
 11  Тип привода                     35677 non-null  object 
 12  Сегмент                         

<br>

Выведем размер таблицы.

In [6]:
# размер таблицы
raw_shape = df.shape
raw_shape

(39966, 25)

Мы имеем $25$ колонок и $39966$ строк.

<br>

Вот список с названиями колонок, где есть пропуски.

In [7]:
# колонки с пропусками
df.columns[df.isna().any()]

Index(['Модификация', 'Год выпуска', 'Вид топлива', 'Объём двиг, л,',
       'Коробка передач', 'Тип привода', 'Сегмент', 'Тип клиента',
       'Форма расчета', 'Количество'],
      dtype='object')

<br>

Выведем типы данных колонок.

In [8]:
df.dtypes

Год                                 int64
Месяц                              object
Компания                           object
Бренд                              object
Модель                             object
Модификация                        object
Год выпуска                        object
Страна-производитель               object
Вид топлива                        object
Объём двиг, л,                     object
Коробка передач                    object
Тип привода                        object
Сегмент                            object
Регион                             object
Наименование дилерского центра     object
Тип клиента                        object
Форма расчета                      object
Количество                        float64
Цена, USD                          object
Продажа, USD                       object
Область                            object
Сегментация 2013                   object
Класс 2013                         object
Сегментация Eng                   

<br>

### Обзор колонок

Чтобы понять, всё ли в порядке с типами данных в таблице, нужно разобрать каждую колонку.\
В этом нам поможет функция, а также документация, где дано описание каждой колонки.

In [9]:
# функция для обзора
def obs(column):
    print(df[column].value_counts(dropna=False))

<br>

`Год` – год продажи.

In [10]:
obs('Год')

2019    39966
Name: Год, dtype: int64


Имеем одно значение, `2019` год, как и было указано во введении. Тип данных верный. 

<br>

`Месяц` – месяц продажи.

In [11]:
obs('Месяц')

Июль        5930
Сентябрь    5312
Август      4812
Май         4798
Июнь        4754
Апрель      4177
Март        3662
Январь      3384
Февраль     3137
Name: Месяц, dtype: int64


Имеем значения с январь по сентябрь, как и было указано во введении. Тип данных верный.

<br>

`Компания` – название автоцентра.

In [12]:
obs('Компания')

Toyota Motor Kazakhstan          11951
Astana Motors                     7395
БИПЭК АВТО                        3624
Вираж                             2909
Astana Motors                     2774
Allur Auto                        2526
Renault Россия                    1796
Nissan Manufacturing RUS          1149
Равон Моторс Казахстан             947
ТК КАМАЗ                           893
Mercur Auto                        639
Ravon Motors Kazakstan             559
Subaru Kazakhstan                  503
ММС Рус                            408
Лифан Моторс Рус                   233
Автоцентр-Бавария                  224
СВС-ТРАНС                          202
Hyundai Com Trans Kazakhstan       155
УзАвто-Казахстан                   152
Автомир ГК                         112
Volkswagen Group Rus               109
Eurasia Motor Premium              102
Almaty Motors Premium               94
СемАЗ                               93
Autokapital                         85
TERRA MOTORS             

Видны неявные дубликаты. Тип данных верный.

<br>

`Бренд` – название продаваемой марки автомобиля.

In [13]:
obs('Бренд')

Toyota                  10745
Hyundai                 10168
Jac                      1991
Lada                     1969
GAZ                      1821
Renault                  1796
Ravon                    1794
Lexus                    1206
Nissan                   1053
UAZ                       927
KAMAZ                     892
Kia                       832
Volkswagen                750
Chevrolet                 635
Subaru                    504
Mitsubishi                410
Skoda                     379
ANKAI                     300
Lifan                     233
BMW                       209
Isuzu                     198
Hyundai Truck & Bus       155
Land Rover                135
Mazda                     112
Infiniti                   96
Mercedes-Benz              94
Cadillac                   72
Audi                       52
Porsche                    52
Yutong                     44
Shacman                    43
Volvo                      42
Hino                       41
Jaguar    

Тип данных верный.

<br>

`Модель` – название модели автомобиля.

In [14]:
obs('Модель')

Camry        4853
Tucson       3271
Accent       2211
Elantra      2062
Creta        1775
             ... 
HD 160          1
CLA-Class       1
6 серия         1
XE              1
iev             1
Name: Модель, Length: 273, dtype: int64


Тип данных верный.

<br>

`Модификация` – модификация модели автомобиля.

In [15]:
obs('Модификация')

NaN                                                           3591
B5                                                            1470
BX                                                            1210
54                                                            1007
Hyundai Elantra AD FL Active                                   764
                                                              ... 
Городской автобус ANKAI модели HFF6850G  MXBTBGKD0KK500406       1
R3 Optimum  М/T                                                  1
DRIVE A/T WMP                                                    1
GAZ ГАЗ-3309-397 грузовой                                        1
Городской автобус ANKAI модели HFF6850G  MXBTBGKD0KK500374       1
Name: Модификация, Length: 1384, dtype: int64


Тип данных верный. Но в комментариях документации нам предлагают избавиться от этой колонки.

<br>

`Год выпуска` – год производства автомобиля.

In [16]:
obs('Год выпуска')

2019     29185
2018      8679
2 019      841
NaN        501
2 018      402
2017       324
2016        30
2014         2
2011         1
2013         1
Name: Год выпуска, dtype: int64


Тип данных неверный. Нужно поменять на `int`.\
Также видны неявные дубликаты.

<br>

`Страна-производитель` – страна, где произведён автомобиль.

In [17]:
obs('Страна-производитель')

Республика Казахстан    19369
Российская Федерация    12846
Япония                   4595
Турция                   1167
Таиланд                   970
США                       289
Германия                  265
UK                        197
Китай                      94
Узбекистан                 70
Австрия                    33
Швеция                     18
Корея                      17
Нидерланды                 12
Испания                     9
Бельгия                     6
Польша                      6
Венгрия                     2
Белоруссия                  1
Name: Страна-производитель, dtype: int64


Тип данных верный.

<br>

`Вид топлива` – бензин, дизель, электричество, гибрид.

In [18]:
obs('Вид топлива')

Бензин           34801
NaN               3140
Дизель            1762
бензин             159
дизель              41
2                   23
1,6                 14
Электричество       10
гибрид              10
0                    3
Электро              3
Name: Вид топлива, dtype: int64


Тип данных верный.\
Но видны неявные дубликаты и аномальные значения. Нужно разобраться.

<br>

`Объём двиг, л,` – объем двигателя автомобиля в литрах.

In [19]:
obs('Объём двиг, л,')

1,6     5337
2       5097
NaN     4258
2,7     3394
2.5     2641
        ... 
3,7        1
4,3,       1
26,7       1
22,7       1
6,7        1
Name: Объём двиг, л,, Length: 115, dtype: int64


Здесь неверный тип данных.\
Видны аномальные значения.

<br>

`Коробка передач` – тип коробки переключения передач.

In [20]:
obs('Коробка передач')

AT               11379
6 AT              8842
MT                3297
NaN               3255
CVT               2100
                 ...  
8 AT                 1
8-ступ АКПП          1
12АТ                 1
Powershift S6        1
7 АКПП PDK           1
Name: Коробка передач, Length: 84, dtype: int64


Тип данных верный.

<br>

`Тип привода` – тип привода автомобиля.

In [21]:
obs('Тип привода')

4WD              9412
2 WD             5078
2WD              4678
FF               4578
NaN              4289
Передний         3270
4 WD             2607
Полный           1917
Задний           1469
FWD               995
AWD               962
RWD               205
Передний (FF)     136
0                 116
передний           56
quattro            47
4х2                38
4Motion            26
4x4                22
4х4                18
4x2                16
4х2.2              10
4X4                 6
полный              5
4X2                 4
#Н/Д                4
Астана              2
Name: Тип привода, dtype: int64


Тип данных верный.\
Но видны неявные дубликаты и аномальные значения.

<br>

`Сегмент` – сегмент, к которому относится авто.

In [22]:
obs('Сегмент')

NaN                        6761
D класс                    4368
C-SUV                      3484
Компактные SUV             2603
B                          2211
                           ... 
5 door                        1
A класс                       1
Крупнотоннажные               1
Sportcar                      1
Промтоварный фургон Т50       1
Name: Сегмент, Length: 120, dtype: int64


Тип данных верный.\
Видны аномальные значения.\
Также в комментариях к документации нам нам предлагают избавиться от этой колонки.

<br>

`Регион` – это город продажи.\
Проверить это можно, сверясь с этим [списком](https://ru.wikipedia.org/wiki/Список_городов_Казахстана_по_численности_населения).

In [23]:
obs('Регион')

Алматы              11179
Нур-Султан           8244
Шымкент              2880
Костанай             2447
Атырау               2162
Караганда            2076
ЭКСПОРТ              1810
Актау                1495
Уральск              1493
Актобе               1351
Павлодар             1214
Усть-Каменогорск     1160
Кызылорда             554
Петропавловск         508
Кокшетау              471
Семей                 290
Талдыкорган           230
Тараз                 205
Экибастуз              88
Рудный                 47
Каскелен               38
Туркестан              15
Сарыагаш                4
Кульсары                2
Риддер                  2
Зыряновск               1
Name: Регион, dtype: int64


Тип данных верный.

<br>

`Наименование дилерского центра` – название автоцентра.

In [24]:
obs('Наименование дилерского центра')

Hyundai Premium Almaty    1600
Hyundai Auto Almaty       1425
Hyundai Premium Astana    1385
Тойота Центр Алматы       1343
Тойота Центр Астана       1333
                          ... 
СемАЗ Кызылорда              1
Меридиан АВТО Актау          1
AST-commerce                 1
КВЦ-АВТО Караганда           1
Ford Атырау                  1
Name: Наименование дилерского центра, Length: 205, dtype: int64


Тип данных верный.\
Но в комментариях к документации нам предлагают избавиться от этой колонки, так как она дублирует колонку `Компания`.

<br>

`Тип клиента` – юридическое или физическое лицо.

In [25]:
obs('Тип клиента')

Физ. Лицо       24568
Юр. Лицо         7873
NaN              7047
Корп. клиент      161
ФизЛицо           132
юридическое        97
ЮрЛицо             30
физ.лицо           29
Юр.Лицо            25
физическое          4
Name: Тип клиента, dtype: int64


Тип данных верный.\
Видны неявные дубликаты.\
Нам предлагают удалить эту колонку, так как для анализа это некритично.

<br>

`Форма расчета` – наличный и безналичный расчет.

In [26]:
obs('Форма расчета')

NaN                                         25928
кредит                                       6190
безналичный                                  4276
Безналичная                                  1335
Собственные средства                          698
наличный                                      599
Наличная                                      499
Безналичная                                   204
Акция Собственные средства (Trade in 5%)       78
нал                                            58
trade-in                                       35
Перечисление                                   21
Без оплаты                                      8
б/н                                             8
перечислением                                   6
Отсрочка платежа                                6
наличные                                        6
Без.Наличный                                    5
лизинг                                          2
безналичный                                     2


Тип данных верный.\
Видны неявные дубликаты.\
Предложено удалить эту колонку, так как много пропусков.

<br>

`Количество` – количество автомобилей в заказе.

In [27]:
obs('Количество')

 1.0      37203
 2.0       1017
 3.0        452
 4.0        282
 5.0        185
 6.0        120
 7.0        100
 8.0         79
 10.0        63
 9.0         61
 12.0        45
 13.0        36
 11.0        35
 14.0        33
 15.0        26
 18.0        24
 16.0        23
 17.0        18
 20.0        15
 19.0        14
 22.0        12
 24.0        12
 23.0        12
 21.0        11
 25.0         9
 26.0         9
 30.0         9
 32.0         6
 NaN          6
 29.0         6
 50.0         5
 27.0         4
 28.0         3
 33.0         3
 31.0         3
 36.0         3
 62.0         2
 43.0         1
 70.0         1
 35.0         1
 60.0         1
 66.0         1
 42.0         1
-1.0          1
 115.0        1
 54.0         1
 100.0        1
 79.0         1
 47.0         1
 34.0         1
 41.0         1
 38.0         1
 63.0         1
 46.0         1
 37.0         1
 51.0         1
 40.0         1
Name: Количество, dtype: int64


Тип данных неверный.\
Возможно, есть аномалия.

<br>

`Цена, USD` – цена автомобиля.

In [28]:
obs('Цена, USD')

9884,695773    559
11600          531
13800          428
12819,73695    412
9893,925137    402
              ... 
15499,17108      1
45351,31154      1
30261,23077      1
60256,41026      1
47109,86737      1
Name: Цена, USD, Length: 3510, dtype: int64


Тип данных неверный.

<br>

`Продажа, USD` – цена заказа (цена авто, умноженная на количество, за вычетом скидок, если есть)

In [29]:
obs('Продажа, USD')

9884,695773    559
12819,73695    412
9893,925137    402
9969,169424    315
13800          230
              ... 
12949,05399      1
18317,43153      1
74431,49359      1
24840,22281      1
46012,02412      1
Name: Продажа, USD, Length: 4183, dtype: int64


Тип данных неверный.

<br>

`Область` – область Казахстана, где продан автомобиль.

In [30]:
obs('Область')

г.Алматы                          11179
г.Нур-Султан                       8244
Южно-Казахстанская область         2880
Костанайская область               2494
Атырауская область                 2164
Карагандинская область             2076
Экспорт область                    1810
Мангистауская область              1495
Западно-Казахстанская область      1493
Восточно-Казахстанская область     1453
Актюбинская область                1351
Павлодарская область               1302
Кызылординская область              554
Северо-Казахстанская область        508
Акмолинская область                 471
Алматинская область                 268
Жамбылская область                  205
Туркестанская область                19
Name: Область, dtype: int64


Тип данных верный.

<br>

`Сегментация 2013` – категории автомобилей.

In [31]:
obs('Сегментация 2013')

Внедорожники               17928
Легковые автомобили        16333
Коммерческие автомобили     4329
Пикапы                       974
Минивэны                     402
Name: Сегментация 2013, dtype: int64


Тип данных верный.

<br>

`Класс 2013` – классификация автомобилей.

In [32]:
obs('Класс 2013')

B класс                      7028
Компактные SUV               6593
D класс                      5385
Субкомпактные SUV            5339
Среднеразмерные SUV          3994
C класс                      3584
Полноразмерные SUV           2002
Малотоннажные грузовики      1940
Крупнотоннажные грузовики    1034
Pick-ups                      974
Микроавтобусы                 433
Компактвэн                    354
Развозные автомобили          352
Большие автобусы              321
E класс                       244
Среднетоннажные грузовики     192
Средние автобусы               57
Полноразмерный Минивэн         48
A класс                        42
F класс                        40
Спортивные автомобили          10
Name: Класс 2013, dtype: int64


Тип данных верный.

<br>

`Сегментация Eng` – английская сегментация.

In [33]:
obs('Сегментация Eng')

K1    11932
B      7028
D      5385
K2     3994
C      3584
K3     2002
N      1940
O      1591
K4      974
L1      429
M       402
L2      369
E       244
A        42
F        40
G2       10
Name: Сегментация Eng, dtype: int64


Тип данных верный.\
Нам предложено избавиться от этой колонки.

<br>

`Локализация производства` – автомобиль домашнего или зарубежного производства.

In [34]:
obs('Локализация производства')

Импорт                    20597
Локальное производство    19369
Name: Локализация производства, dtype: int64


Тип данных верный.\
Нам предложено избавиться от этой колонки.

<br>

Итак, в документации приложены комментарии.\
**В них нам предлагают избавиться от следующих колонок:**
* `Модификация`
* `Сегмент`
* `Наименование дилерского центра` — дублирует колонку `Компания`.
* `Тип клиента` — в рамках анализа некритично.
* `Форма расчета` — много пропусков.
* `Сегментация Eng`
* `Локализация производства` — дублирует колонку `Страна-производитель`.

Но мы сами решим, что с ними делать.

<br>

**Были обнаружены неверные типы данных в следующих колонках:**
1. Год выпуска (`object` преобразовать в `int`)
2. Объём двиг, л, (`object` преобразовать в `float`)
3. Количество (`float` преобразовать в `int`)
4. Цена, USD (`object` преобразовать в `int`)
5. Продажа, USD (`object` преобразовать в `int`)

<br>

**Пожалуй, с обзором данных закончили. Пора приступить к их предобработке.**

<br>

## Предобработка данных

### Работа с колонками

#### Удаление

Решим, какие колонки нам не нужны. 

Однозначно можно избавиться от полей `Модификация` и `Сегментация Eng`, потому что эта информация действительно мало чего несёт.

In [35]:
df.drop(columns=['Модификация', 'Сегментация Eng'], inplace=True)

<br>

Под вопросом стоят следующие колонки:
* `Сегмент` и `Класс 2013`. Они похожи, их нужно сравнить.
* `Наименование дилерского центра`. Она похожа на колонку `Компания`. Нужно сравнить их.
* `Форма расчёта`. Много пропусков. Нужно посмотреть эти пропуски.
* `Сегментация 2013`. С ней нужно глубже разобраться.

<br>

Напишем функцию, показывающую данные колонок.

In [36]:
def column_data(column):
    print(df[column].value_counts(dropna=False).head(20))
    print('')
    print(f'Количество уникальных значений: {df[column].nunique()}')
    print('')
    print(f'Список первых 50 уникальных значений: {df[column].sort_values().unique()[:50]}')

<br>

Итак, **сравнение `Сегмент` и `Класс 2013`**.

In [37]:
column_data('Сегмент')

NaN                            6761
D класс                        4368
C-SUV                          3484
Компактные SUV                 2603
B                              2211
C                              2062
B класс                        1735
Среднеразмерные SUV            1715
B-SUV                          1562
C класс                        1338
Грузовой автомобиль             892
Полноразмерные SUV              808
Пикапы                          778
Среднеразмерные SUV Premium     663
D-SUV                           639
Внедорожники                    617
D класс                         573
Легковые автомобили             547
SUV cars                        504
легковой                        380
Name: Сегмент, dtype: int64

Количество уникальных значений: 119

Список первых 50 уникальных значений: ['#Н/Д' '0' '16+1' '43+1' '5 door' '5 doors' '5doors' 'A класс' 'A класс '
 'ASTER Auto' 'Ambulance' 'B' 'B класс' 'B класс ' 'B-SUV' 'Bus' 'C'
 'C класс' 'C класс ' 'C-SUV' 'C

In [38]:
column_data('Класс 2013')

B класс                      7028
Компактные SUV               6593
D класс                      5385
Субкомпактные SUV            5339
Среднеразмерные SUV          3994
C класс                      3584
Полноразмерные SUV           2002
Малотоннажные грузовики      1940
Крупнотоннажные грузовики    1034
Pick-ups                      974
Микроавтобусы                 433
Компактвэн                    354
Развозные автомобили          352
Большие автобусы              321
E класс                       244
Среднетоннажные грузовики     192
Средние автобусы               57
Полноразмерный Минивэн         48
A класс                        42
F класс                        40
Name: Класс 2013, dtype: int64

Количество уникальных значений: 21

Список первых 50 уникальных значений: ['A класс' 'B класс' 'C класс' 'D класс' 'E класс' 'F класс' 'Pick-ups'
 'Большие автобусы' 'Компактвэн' 'Компактные SUV'
 'Крупнотоннажные грузовики' 'Малотоннажные грузовики' 'Микроавтобусы'
 'Полноразмерные SUV'

В общем, в колонке `Сегмент` много мусорных значений, колонка `Класс 2013` значительно привлекательнее.\
**Сносим первую.**

<br>

**Сравнение `Наименование дилерского центра` и `Компания`**.

In [39]:
column_data('Наименование дилерского центра')

Hyundai Premium Almaty     1600
Hyundai Auto Almaty        1425
Hyundai Premium Astana     1385
Тойота Центр Алматы        1343
Тойота Центр Астана        1333
Toyota City Астана         1319
Toyota City                1258
Тойота Центр Жетысу        1150
HYUNDAI AUTO ASTANA        1117
Allur Auto Almaty          1079
ЭКСПОРТ                     773
Hyundai Center Shimkent     768
Тойота Центр Шымкент        750
Автомир-Центр Караганда     707
Тойота Центр Атырау         634
Caspi Auto Атырау           569
ASTER AUTO Алматы           554
Тойота Центр Костанай       542
Лексус Астана               519
БИПЭК АВТО Астана           492
Name: Наименование дилерского центра, dtype: int64

Количество уникальных значений: 205

Список первых 50 уникальных значений: ['ABS-INVEST' 'ALAN MOTORS' 'ALAN MOTORS\xa0Усть-Каменогорск'
 'AST-commerce' 'ASTER AUTO CЕМЕЙ' 'ASTER AUTO Алматы'
 'ASTER AUTO Шымкент' 'Aktobe Hyundai Motors' 'Alan Motors Trade'
 'Allur Auto Almaty' 'Allur Auto Astana' 'Allur Au

In [40]:
column_data('Компания')

Toyota Motor Kazakhstan          11951
Astana Motors                     7395
БИПЭК АВТО                        3624
Вираж                             2909
Astana Motors                     2774
Allur Auto                        2526
Renault Россия                    1796
Nissan Manufacturing RUS          1149
Равон Моторс Казахстан             947
ТК КАМАЗ                           893
Mercur Auto                        639
Ravon Motors Kazakstan             559
Subaru Kazakhstan                  503
ММС Рус                            408
Лифан Моторс Рус                   233
Автоцентр-Бавария                  224
СВС-ТРАНС                          202
Hyundai Com Trans Kazakhstan       155
УзАвто-Казахстан                   152
Автомир ГК                         112
Name: Компания, dtype: int64

Количество уникальных значений: 40

Список первых 50 уникальных значений: ['Allur Auto' 'Almaty Motors Premium' 'Astana Motors' 'Astana Motors '
 'Autokapital' 'Caspian Motors' 'Daewoo Bus K

<br>

Сводная таблица позволит лучше различить колонки.

In [41]:
# сводная таблица
df.pivot_table(index=['Компания', 'Наименование дилерского центра'], aggfunc='size')\
  .reset_index()\
  .sort_values(by=0, ascending=False)\
  .reset_index(drop=True)\
  .rename(columns={0: 'Кол-во покупок'})\
  .head(20)

,Компания,Наименование дилерского центра,Кол-во покупок
0,Toyota Motor Kazakhstan,Тойота Центр Алматы,1343
1,Toyota Motor Kazakhstan,Тойота Центр Астана,1333
2,Toyota Motor Kazakhstan,Toyota City Астана,1317
3,Toyota Motor Kazakhstan,Toyota City,1258
4,Astana Motors,Hyundai Premium Almaty,1211
5,Toyota Motor Kazakhstan,Тойота Центр Жетысу,1150
6,Astana Motors,Hyundai Premium Astana,1025
7,Allur Auto,Allur Auto Almaty,950
8,Astana Motors,Hyundai Auto Almaty,892
9,Astana Motors,HYUNDAI AUTO ASTANA,860


Думаю, **стоит оставить обе колонки.** У одной компании может быть несколько дилерских центров. Почему бы не оставить эту информацию.

<br>

**Смотрим `Форма расчета`.**

In [42]:
column_data('Форма расчета')

NaN                                         25928
кредит                                       6190
безналичный                                  4276
Безналичная                                  1335
Собственные средства                          698
наличный                                      599
Наличная                                      499
Безналичная                                   204
Акция Собственные средства (Trade in 5%)       78
нал                                            58
trade-in                                       35
Перечисление                                   21
Без оплаты                                      8
б/н                                             8
перечислением                                   6
Отсрочка платежа                                6
наличные                                        6
Без.Наличный                                    5
лизинг                                          2
безналичный                                     2


In [43]:
print(f'Процент пропущенных значений в колонке `Форма расчета`: {round(df["Форма расчета"].isna().mean() * 100)}')

Процент пропущенных значений в колонке `Форма расчета`: 65


Довольно большой процент пропусков, с которыми ничего не поделаешь.\
Но всё равно не хочется избавляться от колонки. **Оставим её.**

<br>

**Смотрим `Сегментация 2013`.**

In [44]:
column_data('Сегментация 2013')

Внедорожники               17928
Легковые автомобили        16333
Коммерческие автомобили     4329
Пикапы                       974
Минивэны                     402
Name: Сегментация 2013, dtype: int64

Количество уникальных значений: 5

Список первых 50 уникальных значений: ['Внедорожники' 'Коммерческие автомобили' 'Легковые автомобили' 'Минивэны'
 'Пикапы']


Данную категоризацию также не хочется убирать из таблицы. **Оставим эту колонку.**

<br>

Таким образом, **сносим только одну мусорную колонку — `Сегмент`.**

In [45]:
# удаление поля
df.drop(columns='Сегмент', inplace=True)

<br>

#### Переименование

Нужно привести все колонки к единому стилю, переименовав их.

Вот их список.

In [46]:
df.columns

Index(['Год', 'Месяц', 'Компания', 'Бренд', 'Модель', 'Год выпуска',
       'Страна-производитель', 'Вид топлива', 'Объём двиг, л,',
       'Коробка передач', 'Тип привода', 'Регион',
       'Наименование дилерского центра', 'Тип клиента', 'Форма расчета',
       'Количество', 'Цена, USD', 'Продажа, USD', 'Область',
       'Сегментация 2013', 'Класс 2013', 'Локализация производства'],
      dtype='object')

In [47]:
# переименование колонок
df = df.rename(columns={'Год': 'year',
                        'Месяц': 'month',
                        'Компания': 'company',
                        'Модель': 'model',
                        'Бренд': 'brand',
                        'Год выпуска': 'manufacturing_year',
                        'Страна-производитель': 'manufacturing_country',
                        'Вид топлива': 'fuel_type',
                        'Объём двиг, л,': 'engine_displacement_l',
                        'Коробка передач': 'transmission_type',
                        'Тип привода': 'drive_type',
                        'Регион': 'city',
                        'Наименование дилерского центра': 'dealer_center_name',
                        'Тип клиента': 'customer_type',
                        'Форма расчета': 'payment_form',
                        'Количество': 'quantity',
                        'Цена, USD': 'price_usd',
                        'Продажа, USD': 'sale_usd',
                        'Область': 'region',
                        'Сегментация 2013': 'car_type',
                        'Класс 2013': 'car_classification',
                        'Локализация производства': 'assembly_type'})

<br>

<br>

#### Создание

Колонки `year` и `month` можно объединить в одну, создав `purchase_date` — колонку с датой приобретения авто.

In [48]:
# словарь с месяцами и их числовыми значениями
month_mapping = {'Январь': 1,
                 'Февраль': 2,
                 'Март': 3,
                 'Апрель': 4,
                 'Май': 5,
                 'Июнь': 6,
                 'Июль': 7,
                 'Август': 8,
                 'Сентябрь': 9}

# замена значений в `month` с использованием словаря
df['month'] = df['month'].map(month_mapping)

# создание колонки `purchase_date` путём объединения `year` и `month`
df['purchase_date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str), format='%Y-%m')

<br>

Можно задать иной, более логичный порядок колонкам в таблице.\
Заодно избавимся от колонок с годом и месяцем.

In [49]:
df = df[['purchase_date',
         'company',
         'car_type',
         'brand',
         'model',
         'car_classification',
         'manufacturing_year',
         'assembly_type',
         'manufacturing_country',
         'fuel_type',
         'engine_displacement_l',
         'transmission_type',
         'drive_type',
         'region',
         'city',
         'dealer_center_name',
         'customer_type',
         'payment_form',
         'quantity',
         'price_usd',
         'sale_usd']]

<br>

#### Замена типов данных

Теперь нужно поменять типы данных в колонках, где они неверны.

Вот колонки и их типы данных.

In [50]:
df.dtypes

purchase_date            datetime64[ns]
company                          object
car_type                         object
brand                            object
model                            object
car_classification               object
manufacturing_year               object
assembly_type                    object
manufacturing_country            object
fuel_type                        object
engine_displacement_l            object
transmission_type                object
drive_type                       object
region                           object
city                             object
dealer_center_name               object
customer_type                    object
payment_form                     object
quantity                        float64
price_usd                        object
sale_usd                         object
dtype: object

**Где неверные типы данных:**
1. `manufacturing_year` — `object` преобразовать в `int`.
2. `engine_displacement_l` — `object` преобразовать в `float`.
3. `quantity` — `float` преобразовать в `int`.
4. `price_usd` — `object` преобразовать в `int`.
5. `sale_usd` — `object` преобразовать в `int`.

<br>

Функция для анализа колонок с неверными типами данных.

In [51]:
def dtype_change(column):
    print(df[column].sort_values().unique())
    print('')
    print(f'Количество уникальных значений: {df[column].nunique()}')

<br>

##### `Год производства`

Посмотрим сначала уникальные значения внутри.

In [52]:
dtype_change('manufacturing_year')

['2011' '2013' '2014' '2016' '2017' '2018' '2019' '2\xa0018' '2\xa0019'
 nan]

Количество уникальных значений: 9


Странные символы `2\xa0` — результат работы кодировки.\
Они скрывают неявные дубликаты, они не дадут преобразовать строку в целочисленный формат.\
Избавимся от этих дублей, убьём сразу двух зайцев.

In [53]:
# устранение дублей
df['manufacturing_year'] = df['manufacturing_year'].replace('2\xa0018', '2018')\
                                                   .replace('2\xa0019', '2019')

In [54]:
# проверка
dtype_change('manufacturing_year')

['2011' '2013' '2014' '2016' '2017' '2018' '2019' nan]

Количество уникальных значений: 7


Дубли устранились.

Поменяем тип данных.

In [55]:
# смена типа данных
df['manufacturing_year'] = pd.to_numeric(df['manufacturing_year'], errors='coerce').astype('Int64')

In [56]:
# проверка
df['manufacturing_year'].dtypes

Int64Dtype()

Всё верно.

<br>

##### `Объём двигателя`

Посмотрим сначала уникальные значения внутри.

In [57]:
dtype_change('engine_displacement_l')

['#Н/Д' '0' '1,2' '1,248' '1,2T' '1,33' '1,4' '1,4 Turbo' '1,485' '1,5'
 '1,591' '1,596' '1,598' '1,6' '1,6 MPI' '1,6 T-GDI' '1,69' '1,7' '1,774'
 '1,8' '1,998' '1,999' '1.5' '1.6' '10,5' '10,7' '11' '11,7' '12' '12,3'
 '12,7' '12,8' '13' '13,7' '14,7' '15,7' '16,7' '17,7' '18,7' '19,7' '2'
 '2,0' '2,0 CRDI' '2,0 MPI' '2,2' '2,359' '2,4' '2,4 GDI' '2,4 MPI' '2,4G'
 '2,5' '2,5 CRDI VGT' '2,5 CRDI WGT' '2,693' '2,7' '2,8' '2,9' '2.0'
 '2.0h' '2.4G' '2.5' '2.7' '2.7 ' '2.8' '20,7' '21,7' '22,7' '23,7' '24,7'
 '25,7' '26,7' '3' '3,0 L' '3,342' '3,47' '3,5' '3,6' '3,7' '3,8' '3,9'
 '3.5' '3.8' '4' '4,3' '4,3,' '4,4' '4,6' '4,7' '4,9' '4,98' '4,98 L,'
 '4.0' '4.6' '400 Л.С.' '5' '5,2' '5,5' '5,6' '5,7' '5.7' '6,2' '6,5'
 '6,6' '6,7' '6,7L' '7,5' '7,6' '7,7' '8,4 L,' '8,7' '88 KWH' '9,7' 'AT'
 'MT' nan]

Количество уникальных значений: 114


Непорядок. Запятые в объёме двигателя нужно поменять на точки, а от всех буквенных значений нужно избавиться. Иначе не сможем преобразовать во `float`.

In [58]:
# замена запятых на точки
df['engine_displacement_l'] = df['engine_displacement_l'].str.replace(',', '.')\
                                                         .str.replace('[^0-9.]', '', regex=True)

<br>

Посмотрим, что поменялось.

In [59]:
dtype_change('engine_displacement_l')

['' '0' '1.2' '1.248' '1.33' '1.4' '1.485' '1.5' '1.591' '1.596' '1.598'
 '1.6' '1.69' '1.7' '1.774' '1.8' '1.998' '1.999' '10.5' '10.7' '11'
 '11.7' '12' '12.3' '12.7' '12.8' '13' '13.7' '14.7' '15.7' '16.7' '17.7'
 '18.7' '19.7' '2' '2.0' '2.2' '2.359' '2.4' '2.5' '2.693' '2.7' '2.8'
 '2.9' '20.7' '21.7' '22.7' '23.7' '24.7' '25.7' '26.7' '3' '3.0' '3.342'
 '3.47' '3.5' '3.6' '3.7' '3.8' '3.9' '4' '4.0' '4.3' '4.3.' '4.4' '4.6'
 '4.7' '4.9' '4.98' '4.98.' '400..' '5' '5.2' '5.5' '5.6' '5.7' '6.2'
 '6.5' '6.6' '6.7' '7.5' '7.6' '7.7' '8.4.' '8.7' '88' '9.7' nan]

Количество уникальных значений: 87


Диапазон уникальных значений уменьшился.\
Значения, которые состояли полностью из букв, поменялись на пустые значения. Их нужно будет поменять на пропуски.\
В некоторых значениях появились лишние точки. Нужно тоже исправить.

In [60]:
# strip удаляет первый и последний символ в строке, указанный в методе
df['engine_displacement_l'] = df['engine_displacement_l'].str.strip('.')\
                                                         .replace('', np.nan)

<br>

Результат.

In [61]:
dtype_change('engine_displacement_l')

['0' '1.2' '1.248' '1.33' '1.4' '1.485' '1.5' '1.591' '1.596' '1.598'
 '1.6' '1.69' '1.7' '1.774' '1.8' '1.998' '1.999' '10.5' '10.7' '11'
 '11.7' '12' '12.3' '12.7' '12.8' '13' '13.7' '14.7' '15.7' '16.7' '17.7'
 '18.7' '19.7' '2' '2.0' '2.2' '2.359' '2.4' '2.5' '2.693' '2.7' '2.8'
 '2.9' '20.7' '21.7' '22.7' '23.7' '24.7' '25.7' '26.7' '3' '3.0' '3.342'
 '3.47' '3.5' '3.6' '3.7' '3.8' '3.9' '4' '4.0' '4.3' '4.4' '4.6' '4.7'
 '4.9' '4.98' '400' '5' '5.2' '5.5' '5.6' '5.7' '6.2' '6.5' '6.6' '6.7'
 '7.5' '7.6' '7.7' '8.4' '8.7' '88' '9.7' nan]

Количество уникальных значений: 84


<br>

Наконец, поменяем тип данных.

In [62]:
# смена типа данных
df['engine_displacement_l'] = df['engine_displacement_l'].astype('float')

In [63]:
# проверка
df['engine_displacement_l'].dtypes

dtype('float64')

Всё верно.

<br>

##### `Количество авто в заказе`

Посмотрим сначала уникальные значения внутри.

In [64]:
dtype_change('quantity')

[ -1.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  40.  41.  42.
  43.  46.  47.  50.  51.  54.  60.  62.  63.  66.  70.  79. 100. 115.
  nan]

Количество уникальных значений: 56


Здесь всё просто. Меняем тип данных.

In [65]:
# замена типа данных
df['quantity'] = df['quantity'].astype('Int64')

In [66]:
# проверка
df['quantity'].dtype

Int64Dtype()

Всё верно.

<br>

##### `Стоимость автомобиля`

Посмотрим сначала уникальные значения внутри.

In [67]:
dtype_change('price_usd')

['100056,4835' '10015,24362' '100216,6313' ... '99467,01183' '9969,169424'
 '99767,25773']

Количество уникальных значений: 3510


Здесь с максимальной точностью указана цена автомобиля. Но эта ловля центов нам не нужна. Нужно округлить просто округлить цену.\
Чтобы преобразовать тип данных, сделаем следующее:

In [68]:
# замена разделения дробной части от целого с запятой на точку
df['price_usd'] = df['price_usd'].str.replace(',', '.')

# преобразование строчного типа в вещественный
df['price_usd'] = df['price_usd'].astype('float')

# округление цены автомобиля в большую сторону
df['price_usd'] = df['price_usd'].round()

# преобразование вещественного типа в целочисленный
df['price_usd'] = df['price_usd'].astype('int')

# проверка
df['price_usd'].dtype

dtype('int32')

Всё верно.

<br>

##### `Цена заказа в $`

Посмотрим сначала уникальные значения внутри.

In [69]:
dtype_change('sale_usd')

['-35588,25' '0' '100000' ... '9969,169424' '99766,70464' '99767,25773']

Количество уникальных значений: 4183


Напомню, что это цена автомобиля, умноженная на их количество, за вычетом скидок.

Опять же имеем максимально точную стоимость "заказа". Это нам ни к чему.\
Процесс преобразования такой же, как и в `price_usd`:\
заменим запятую на точку > строчный преобразуем в вещественный > округлим > вещественный преобразуем в целочисленный.

In [70]:
# замена символов
df['sale_usd'] = df['sale_usd'].str.replace(',', '.')

# str > float
df['sale_usd'] = df['sale_usd'].astype('float')

# округление 
df['sale_usd'] = df['sale_usd'].round()

# float > int
df['sale_usd'] = df['sale_usd'].astype('int')

# проверка
df['sale_usd'].dtype

dtype('int32')

Тип данных поменялся.

<br>

**Таким образом, мы поменяли тип данных в следующих колонках:**
1. `manufacturing_year`
2. `engine_displacement_l`
3. `quantity`
4. `price_usd`
5. `sale_usd`

<br>

### "Олицетворение" данных

Как было уже упомянуто, данные в таблице обезличены, не хватает первичных ключей.\
Если мы произведём проверку на явные дубликаты, мы обнаружим очень большое их количество.\
Это такие дубликаты-строки, которые повторяются в полном объёме: мы можем обнаружить абсолютно одинаковые строки.\
Допустим, сейчас выведу строку-клон, у которой уже есть точно такая же строка в таблице.

In [71]:
df[df.duplicated()].head(1)

,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
8,2019-06-01,Mercur Auto,Легковые автомобили,Audi,A6,E класс,2019,Импорт,Германия,Бензин,3.0,S-tronic,quattro,г.Алматы,Алматы,Mercur Auto Алматы,Физ. Лицо,наличный,1,78940,78940


In [72]:
print(f'Количество явных (полных) дубликатов: {df.duplicated().sum()}')
print(f'Процент явных (полных) дубликатов: {round(df.duplicated().sum() / len(df) * 100, 1)}%')

Количество явных (полных) дубликатов: 20197
Процент явных (полных) дубликатов: 50.5%


Удаление их будет серьёзной ошибкой.\
Здесь просто нужно добавить уникальный идентификатор каждой строке, и всё наладится.

<br>

Но перед этим **зададим сортировку сначала по дате покупки, затем по бренду, и затем по модели.**

In [73]:
# сортировка по дате покупки, бренду, модели
df = df.sort_values(by=['purchase_date', 'brand', 'model'])

<br>

Теперь прилепим уникальный ID к каждой строке.

In [74]:
# олицетворение строк
df = df.reset_index(drop=True)\
       .reset_index()\
       .rename(columns={'index': 'id'})

<br>

Теперь проверим, что с явными дубликатами.

In [75]:
print(f'Количество явных (полных) дубликатов: {df.duplicated().sum()}')
print(f'Процент явных (полных) дубликатов: {round(df.duplicated().sum() / len(df) * 100, 1)}%')

Количество явных (полных) дубликатов: 0
Процент явных (полных) дубликатов: 0.0%


Их нет.\
Конечно, данный способ неидеален. Потому что, в теории, один клиент может зайти в разные месяцы и купить автомобиль снова.\
Но это будет очень состоятельный покупатель и такое явление очень большая редкость. Получается, что это клиент с одним и тем же ID.\
Но это совсем незначительная трудность.

<br>

### Дубликаты и аномалии

Проблему явных дубликатов мы исправили.\
Теперь нужно разобраться с неявными дубликатами и аномальными значениями.\
Для этого пройдёмся по каждой колонке.

<br>

Создадим функцию для анализа колонки.

In [76]:
def duplicates(column):
    print(df[column].value_counts(dropna=False).head(10))
    print('')
    print(f'Количество непропущенных значений: {df[column].count()}')
    print(f'Количество уникальных значений: {df[column].nunique()}')
    print('')
    print(f'Первые 50 уникальных значений в алфавитном порядке:')
    print(df[column].sort_values().unique()[:50])

<br>

`ID`

In [77]:
duplicates('id')

0        1
35492    1
25241    1
31386    1
29339    1
19100    1
17053    1
23198    1
21151    1
33445    1
Name: id, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 39966

Первые 50 уникальных значений в алфавитном порядке:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]


Ранее каждой строке мы поставили уникальный ID.\
Дубликатов и аномалий нет.

<br>

`Дата покупки`

In [78]:
duplicates('purchase_date')

2019-07-01    5930
2019-09-01    5312
2019-08-01    4812
2019-05-01    4798
2019-06-01    4754
2019-04-01    4177
2019-03-01    3662
2019-01-01    3384
2019-02-01    3137
Name: purchase_date, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 9

Первые 50 уникальных значений в алфавитном порядке:
['2019-01-01T00:00:00.000000000' '2019-02-01T00:00:00.000000000'
 '2019-03-01T00:00:00.000000000' '2019-04-01T00:00:00.000000000'
 '2019-05-01T00:00:00.000000000' '2019-06-01T00:00:00.000000000'
 '2019-07-01T00:00:00.000000000' '2019-08-01T00:00:00.000000000'
 '2019-09-01T00:00:00.000000000']


Все строки показывают $2019$ год и нужные месяца, как положено.\
Дубликатов и аномалий нет.

<br>

##### `Компания`

In [79]:
duplicates('company')

Toyota Motor Kazakhstan     11951
Astana Motors                7395
БИПЭК АВТО                   3624
Вираж                        2909
Astana Motors                2774
Allur Auto                   2526
Renault Россия               1796
Nissan Manufacturing RUS     1149
Равон Моторс Казахстан        947
ТК КАМАЗ                      893
Name: company, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 40

Первые 50 уникальных значений в алфавитном порядке:
['Allur Auto' 'Almaty Motors Premium' 'Astana Motors' 'Astana Motors '
 'Autokapital' 'Caspian Motors' 'Daewoo Bus Kazakhstan'
 'Eurasia Motor Premium' 'Hino Motors ' 'Hyundai Com Trans Kazakhstan '
 'MAN Truck & Bus Kazakhstan' 'MMC RUS' 'Mercur Auto' 'Mercur Autos'
 'Nissan Manufacturing RUS' 'Ravon Motors Kazakstan' 'Renault Россия'
 'Scandinavian Motors' 'Scania Central Asia' 'Subaru Kazakhstan'
 'TERRA MOTORS' 'Toyota Motor Kazakhstan' 'Volkswagen Group Rus'
 'Автодом Motors KST' 'Автокапитал

Выведем полное распределение значений колонки.\
Это поможет найти выбивающиеся значения.

In [80]:
df['company'].value_counts()

Toyota Motor Kazakhstan          11951
Astana Motors                     7395
БИПЭК АВТО                        3624
Вираж                             2909
Astana Motors                     2774
Allur Auto                        2526
Renault Россия                    1796
Nissan Manufacturing RUS          1149
Равон Моторс Казахстан             947
ТК КАМАЗ                           893
Mercur Auto                        639
Ravon Motors Kazakstan             559
Subaru Kazakhstan                  503
ММС Рус                            408
Лифан Моторс Рус                   233
Автоцентр-Бавария                  224
СВС-ТРАНС                          202
Hyundai Com Trans Kazakhstan       155
УзАвто-Казахстан                   152
Автомир ГК                         112
Volkswagen Group Rus               109
Eurasia Motor Premium              102
Almaty Motors Premium               94
СемАЗ                               93
Autokapital                         85
TERRA MOTORS             

Не без помощи поисковых систем замечены неявные дубликаты. Устраним их.

In [81]:
# неявные дубликаты
df['company'] = df['company'].replace('Каспиан Моторс', 'Caspian Motors')\
                             .replace('Astana Motors ', 'Astana Motors')\
                             .replace('Хино Моторс Казахстан', 'Hino Motors')\
                             .replace('MMC RUS', 'ММС Рус')\
                             .replace('Mercur Autos', 'Mercur Auto')\
                             .replace('Равон Моторс Казахстан', 'Ravon Motors Kazakstan')\
                             .replace('Автокапитал', 'Autokapital')\
                             .replace('Автомир ГК', 'Автомир-Центр')

# неточности в названиях
df['company'] = df['company'].replace('Hino Motors ', 'Hino Motors')\
                             .replace('ТОО "Eurasia Motor Zhaik"', 'Eurasia Motor Zhaik')

Посмотрим снова картину.

In [82]:
duplicates('company')

Toyota Motor Kazakhstan     11951
Astana Motors               10169
БИПЭК АВТО                   3624
Вираж                        2909
Allur Auto                   2526
Renault Россия               1796
Ravon Motors Kazakstan       1506
Nissan Manufacturing RUS     1149
ТК КАМАЗ                      893
Mercur Auto                   643
Name: company, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 32

Первые 50 уникальных значений в алфавитном порядке:
['Allur Auto' 'Almaty Motors Premium' 'Astana Motors' 'Autokapital'
 'Caspian Motors' 'Daewoo Bus Kazakhstan' 'Eurasia Motor Premium'
 'Eurasia Motor Zhaik' 'Hino Motors' 'Hyundai Com Trans Kazakhstan '
 'MAN Truck & Bus Kazakhstan' 'Mercur Auto' 'Nissan Manufacturing RUS'
 'Ravon Motors Kazakstan' 'Renault Россия' 'Scandinavian Motors'
 'Scania Central Asia' 'Subaru Kazakhstan' 'TERRA MOTORS'
 'Toyota Motor Kazakhstan' 'Volkswagen Group Rus' 'Автодом Motors KST'
 'Автомир-Центр' 'Автоцентр-Бавари

Теперь стало на $8$ уникальных значений меньше. Дубли устранились.

<br>

`Тип автомобиля`

In [83]:
duplicates('car_type')

Внедорожники               17928
Легковые автомобили        16333
Коммерческие автомобили     4329
Пикапы                       974
Минивэны                     402
Name: car_type, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 5

Первые 50 уникальных значений в алфавитном порядке:
['Внедорожники' 'Коммерческие автомобили' 'Легковые автомобили' 'Минивэны'
 'Пикапы']


Здесь всё интересно намешано.

**Внедорожники** — это легковые или полугрузовые автомобили повышенной проходимости и повышенного комфорта в салоне. По-другому их можно назвать кроссоверами.\
**Легковые автомобили** — это тип автомобиля, который имеют различные кузова: седан, хэтчбек, универсал, лифтбек, кабриолет, купе.\
**Коммерческие автомобили** — это автомобили, используемые в коммерческих целях: грузовики, автобусы и прочее.\
**Пикапы** — это внедорожники, которые могут быть как грузовыми, так и легковыми.\
**Минивэны** — это тип кузова легковых автомобилей с тремя рядами сидений.

Здесь стоит **пикапы перенести во внедорожники, а минивэны в легковые.**

In [84]:
df['car_type'] = df['car_type'].replace('Пикапы', 'Внедорожники')\
                               .replace('Минивэны', 'Легковые автомобили')

Проверим снова.

In [85]:
duplicates('car_type')

Внедорожники               18902
Легковые автомобили        16735
Коммерческие автомобили     4329
Name: car_type, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 3

Первые 50 уникальных значений в алфавитном порядке:
['Внедорожники' 'Коммерческие автомобили' 'Легковые автомобили']


Теперь порядок.

<br>

`Марка автомобиля`

In [86]:
duplicates('brand')

Toyota     10745
Hyundai    10168
Jac         1991
Lada        1969
GAZ         1821
Renault     1796
Ravon       1794
Lexus       1206
Nissan      1053
UAZ          927
Name: brand, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 46

Первые 50 уникальных значений в алфавитном порядке:
['ANKAI' 'Audi' 'BMW' 'Cadillac' 'Chevrolet' 'Daewoo' 'Dong Feng' 'Ford'
 'Foton' 'GAZ' 'Hino' 'Hyundai' 'Hyundai Truck & Bus' 'Infiniti' 'Isuzu'
 'Iveco' 'Jac' 'Jaguar' 'KAMAZ' 'Kia' 'Lada' 'Land Rover' 'Lexus' 'Lifan'
 'MAN' 'Mazda' 'Mercedes-Benz' 'Mercedes-Benz Trucks' 'Mini' 'Mitsubishi'
 'Nefaz' 'Nissan' 'Peugeot' 'Porsche' 'Ravon' 'Renault' 'Scania' 'Shacman'
 'Skoda' 'Subaru' 'Toyota' 'UAZ' 'Volkswagen' 'Volvo' 'Yutong' 'Урал']


Здесь всё в норме. Дублей и аномалий нет.

<br>

`Модель автомобиля`

In [87]:
duplicates('model')

Camry       4853
Tucson      3271
Accent      2211
Elantra     2062
Creta       1775
LC Prado    1711
Nexia R3    1699
S3          1286
RAV4        1213
Corolla     1089
Name: model, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 273

Первые 50 уникальных значений в алфавитном порядке:
['2206' '2217' '2310' '2705' '2752' '3' '3 серия' '300' '301' '3151'
 '3221' '32551' '3302' '3303' '3308' '3309' '3741' '3909' '3962' '4308'
 '4311' '43118' '4320' '4320-1951-40' '4320-1951-60' '43253' '43502'
 '44108' '45141' '45142' '45143' '4x4' '5 серия' '500' '5299' '53215'
 '53504' '53605' '54115' '5490' '5557' '58815Z' '6' '6 серия' '65111'
 '65115' '65116' '65117' '6520' '65206']


<br>

Нужно сделать сводную таблицу по бренду и моделям, чтобы было удобнее разобраться.

In [88]:
# сводная таблица
pt_models = df.pivot_table(index=['brand', 'model'], aggfunc='size').reset_index(name='orders').sort_values(by=['brand', 'orders'], ascending=[True, False])

# раскомментируйте код ниже, чтобы посмотреть всю таблицу
# pd.set_option('display.max_rows', 300)
pt_models

# обязательно раскомментируйте код ниже, чтобы вернуть компактную таблицу
pd.reset_option('display.max_rows')
pt_models

,brand,model,orders
3,ANKAI,HFF6850G,240
0,ANKAI,HF-D105,30
2,ANKAI,HFF6127GZ-4,20
1,ANKAI,HFF6124G03EV3,10
11,Audi,Q8,20
...,...,...,...
271,Урал,4320,7
274,Урал,5557,2
270,Урал,32551,1
272,Урал,4320-1951-40,1


Здесь всё выглядит в порядке.\
Неявных дубликатов и аномалий не видно.\
Но на всякий случай избавимся от возможных пробелов в конце и начале названий.

In [89]:
# устранить лишние пробелы
df['model'] = df['model'].str.strip(' ')

<br>

`Классификация автомобиля`

In [90]:
duplicates('car_classification')

B класс                      7028
Компактные SUV               6593
D класс                      5385
Субкомпактные SUV            5339
Среднеразмерные SUV          3994
C класс                      3584
Полноразмерные SUV           2002
Малотоннажные грузовики      1940
Крупнотоннажные грузовики    1034
Pick-ups                      974
Name: car_classification, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 21

Первые 50 уникальных значений в алфавитном порядке:
['A класс' 'B класс' 'C класс' 'D класс' 'E класс' 'F класс' 'Pick-ups'
 'Большие автобусы' 'Компактвэн' 'Компактные SUV'
 'Крупнотоннажные грузовики' 'Малотоннажные грузовики' 'Микроавтобусы'
 'Полноразмерные SUV' 'Полноразмерный Минивэн' 'Развозные автомобили'
 'Спортивные автомобили' 'Среднеразмерные SUV' 'Среднетоннажные грузовики'
 'Средние автобусы' 'Субкомпактные SUV']


Здесь всё аккуратно. Дублей и аномалий нет.

<br>

`Год производства автомобиля`

In [91]:
duplicates('manufacturing_year')

2019    30026
2018     9081
NaN       501
2017      324
2016       30
2014        2
2011        1
2013        1
Name: manufacturing_year, dtype: Int64

Количество непропущенных значений: 39465
Количество уникальных значений: 7

Первые 50 уникальных значений в алфавитном порядке:
<IntegerArray>
[2011, 2013, 2014, 2016, 2017, 2018, 2019, <NA>]
Length: 8, dtype: Int64


Кажется, здесь всё в порядке. Один автомобиль был даже произведён в $2011$ году. Ничего особого.

<br>

`Где собран автомобиль`

In [92]:
duplicates('assembly_type')

Импорт                    20597
Локальное производство    19369
Name: assembly_type, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 2

Первые 50 уникальных значений в алфавитном порядке:
['Импорт' 'Локальное производство']


Всё в порядке. Есть местная сборка, есть зарубежная сборка.\
Можно проверить это следующим кодом. Результат должен быть таким же, какой выдаёт `Локальное производство`.

In [93]:
len(df[df['manufacturing_country'] == 'Республика Казахстан'])

19369

Сделано в Казахстане, значит локальное производство.

<br>

`Страна сборки`

In [94]:
duplicates('manufacturing_country')

Республика Казахстан    19369
Российская Федерация    12846
Япония                   4595
Турция                   1167
Таиланд                   970
США                       289
Германия                  265
UK                        197
Китай                      94
Узбекистан                 70
Name: manufacturing_country, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 19

Первые 50 уникальных значений в алфавитном порядке:
['UK' 'Австрия' 'Белоруссия' 'Бельгия' 'Венгрия' 'Германия' 'Испания'
 'Китай' 'Корея' 'Нидерланды' 'Польша' 'Республика Казахстан'
 'Российская Федерация' 'США' 'Таиланд' 'Турция' 'Узбекистан' 'Швеция'
 'Япония']


Можно чуть поправить названия некоторых стран.

In [95]:
df['manufacturing_country'] = df['manufacturing_country'].replace({'Республика Казахстан': 'Казахстан',
                                                                   'Российская Федерация': 'Россия',
                                                                   'UK': 'Великобритания'})

В остальном — порядок.

<br>

##### `Тип топлива`

In [96]:
duplicates('fuel_type')

Бензин           34801
NaN               3140
Дизель            1762
бензин             159
дизель              41
2                   23
1,6                 14
Электричество       10
гибрид              10
0                    3
Name: fuel_type, dtype: int64

Количество непропущенных значений: 36826
Количество уникальных значений: 10

Первые 50 уникальных значений в алфавитном порядке:
['0' '1,6' '2' 'Бензин' 'Дизель' 'Электричество' 'Электро' 'бензин '
 'гибрид' 'дизель ' nan]


Здесь нужно привести значения к нужному регистру, а также убрать пробелы, так мы избавимся от дублей.\
Также тут затесались объёмы двигателей, это аномальные значения, их нужно заменить.

In [97]:
df.query('fuel_type == "2" or fuel_type == "1,6" or fuel_type == "0"').head(10)

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
1898,1898,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2019,Импорт,Россия,2,NaN,4WD,4WD,Западно-Казахстанская область,Уральск,Урал-Кров Авто,NaN,NaN,1,12250,12250
1899,1899,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,г.Нур-Султан,Нур-Султан,Кристалл Авто Астана,NaN,NaN,1,12250,12250
1900,1900,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,г.Нур-Султан,Нур-Султан,Кристалл Авто Астана,NaN,NaN,1,12250,12250
1901,1901,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,Кызылординская область,Кызылорда,Кристалл Авто Кызылорда,NaN,NaN,1,12250,12250
1902,1902,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,Карагандинская область,Караганда,Кристалл Авто Караганда,NaN,NaN,1,12250,12250
1903,1903,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,"1,6",NaN,4WD,4WD,Карагандинская область,Караганда,Кристалл Авто Караганда,NaN,NaN,1,12250,12250
1904,1904,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,Карагандинская область,Караганда,Кристалл Авто Караганда,NaN,NaN,1,12250,12250
1905,1905,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,Карагандинская область,Караганда,Кристалл Авто Караганда,NaN,NaN,1,12250,12250
1906,1906,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,Карагандинская область,Караганда,Кристалл Авто Караганда,NaN,NaN,1,12250,12250
1907,1907,2019-01-01,Renault Россия,Внедорожники,Renault,Duster,Субкомпактные SUV,2018,Импорт,Россия,2,NaN,4WD,4WD,Карагандинская область,Караганда,Кристалл Авто Караганда,NaN,NaN,1,12250,12250


Эти числа являются объёмом двигателей, это подтвердит автомобильный интернет портал, их нужно перенести в другую колонку.\
Также выяснил, что тип топлива — бензин должен быть.

<br>

Переместим значения.

In [98]:
# срез
fuel_mist = df.query('fuel_type == "2" or fuel_type == "1,6"')

# замена символов
fuel_mist = fuel_mist['fuel_type'].str.replace(',', '.')

# преобразование в float
fuel_mist = fuel_mist.astype('float')

# в df выбираются строки с индексами из fuel_mist, чтобы заменить значения в `engine_displacement_l` на значения из fuel_mist
df.loc[fuel_mist.index, 'engine_displacement_l'] = fuel_mist

<br>

Приведём символы к нужному регистру.

In [99]:
# сделать заглавной только первую букву
df['fuel_type'] = df['fuel_type'].str.capitalize()

# устранение пробелов и аномальных значений
df['fuel_type'] = df['fuel_type'].str.replace(' ', '')\
                                 .replace('Электро', 'Электричество')\
                                 .replace('2', 'Бензин')\
                                 .replace('1,6', 'Бензин')\
                                 .replace('0', 'Бензин')

<br>

Проверим, что изменилось.

In [100]:
duplicates('fuel_type')

Бензин           35000
NaN               3140
Дизель            1803
Электричество       13
Гибрид              10
Name: fuel_type, dtype: int64

Количество непропущенных значений: 36826
Количество уникальных значений: 4

Первые 50 уникальных значений в алфавитном порядке:
['Бензин' 'Гибрид' 'Дизель' 'Электричество' nan]


Теперь порядок. Аномалии и дубли устранены.

<br>

##### `Объём двигателя`

In [101]:
duplicates('engine_displacement_l')

2.000    7934
1.600    7778
2.500    5034
2.700    4669
NaN      4262
1.500    1423
1.596    1120
2.400    1103
4.600     754
3.500     705
Name: engine_displacement_l, dtype: int64

Количество непропущенных значений: 35704
Количество уникальных значений: 80

Первые 50 уникальных значений в алфавитном порядке:
[1.2   1.248 1.33  1.4   1.485 1.5   1.591 1.596 1.598 1.6   1.69  1.7
 1.774 1.8   1.998 1.999 2.    2.2   2.359 2.4   2.5   2.693 2.7   2.8
 2.9   3.    3.342 3.47  3.5   3.6   3.7   3.8   3.9   4.    4.3   4.4
 4.6   4.7   4.9   4.98  5.    5.2   5.5   5.6   5.7   6.2   6.5   6.6
 6.7   7.5  ]


Когда мы меняли тип данных в этой колонке, мы хорошо предобработали её. Но она до сих пор не в идеальном состоянии.\
Среди значений видим такие, где есть сотые и тысячные доли. Скорее всего, это так перевели кубические сантиметры в литры. **Всё нужно округлить до десятых.**

In [102]:
# округление
df['engine_displacement_l'] = df['engine_displacement_l'].round(1)

<br>

Насколько я помню, были аномалии. Нужно отсортировать уникальные значения в обратном порядке и посмотреть их.

In [103]:
df['engine_displacement_l'].sort_values(ascending=False).unique()

array([400. ,  88. ,  26.7,  25.7,  24.7,  23.7,  22.7,  21.7,  20.7,
        19.7,  18.7,  17.7,  16.7,  15.7,  14.7,  13.7,  13. ,  12.8,
        12.7,  12.3,  12. ,  11.7,  11. ,  10.7,  10.5,   9.7,   8.7,
         8.4,   7.7,   7.6,   7.5,   6.7,   6.6,   6.5,   6.2,   5.7,
         5.6,   5.5,   5.2,   5. ,   4.9,   4.7,   4.6,   4.4,   4.3,
         4. ,   3.9,   3.8,   3.7,   3.6,   3.5,   3.3,   3. ,   2.9,
         2.8,   2.7,   2.5,   2.4,   2.2,   2. ,   1.8,   1.7,   1.6,
         1.5,   1.4,   1.3,   1.2,   nan])

Сложно представить себе автомобиль с объёмом двигателя больше $10$ литров.\
Можно разобрать автомобили с такими значениями.

In [104]:
# авто с двигателями более 10 л
df[df['engine_displacement_l'] >= 10].sort_values(by='engine_displacement_l', ascending=False).head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
11754,11754,2019-04-01,Almaty Motors Premium,Внедорожники,Jaguar,I-Pace,Компактные SUV,2019,Импорт,Великобритания,Электричество,400.0,РЕДУКТОР,4WD,г.Алматы,Алматы,Almaty Motors Premium,Физ. Лицо,наличный,1,105389,105389
4471,4471,2019-02-01,Almaty Motors Premium,Внедорожники,Jaguar,I-Pace,Компактные SUV,2019,Импорт,Великобритания,Электричество,400.0,РЕДУКТОР,4WD,г.Алматы,Алматы,Almaty Motors Premium,Физ. Лицо,наличный,1,117237,117237
21118,21118,2019-06-01,Almaty Motors Premium,Внедорожники,Jaguar,I-Pace,Компактные SUV,2019,Импорт,Великобритания,Электричество,400.0,РЕДУКТОР,4WD,г.Алматы,Алматы,Almaty Motors Premium,Юр. Лицо,безналичный,1,120413,120413
23917,23917,2019-07-01,Allur Auto,Коммерческие автомобили,ANKAI,HFF6124G03EV3,Большие автобусы,2019,Локальное производство,Казахстан,Электричество,88.0,NaN,2WD,Костанайская область,Костанай,Allur Auto Almaty,Юр. Лицо,безналичный,1,307986,307986
23919,23919,2019-07-01,Allur Auto,Коммерческие автомобили,ANKAI,HFF6124G03EV3,Большие автобусы,2019,Локальное производство,Казахстан,Электричество,88.0,NaN,2WD,Костанайская область,Костанай,Allur Auto Almaty,Юр. Лицо,безналичный,1,307986,307986


В самом верху с объёмом $88$ и выше — это электрокары. И там указана мощность вместо объёма.\
Но у электрокаров нет такого понятия как объём двигателя. У них есть ёмкость батареи.\
Им нужно поставить пропущенные значения.

Нива почувствовала себя супермощной: от $13.7$ до $26.7$ литров значение объёма двигателя. Какой вздор.\
Посмотрим распределение объёмов у всех Нив в датасете.

In [105]:
df[df['model'] == 'Niva']['engine_displacement_l'].value_counts()

1.7     542
2.7       1
21.7      1
14.7      1
8.7       1
16.7      1
5.7       1
7.7       1
25.7      1
26.7      1
22.7      1
10.7      1
11.7      1
20.7      1
18.7      1
13.7      1
23.7      1
12.7      1
9.7       1
24.7      1
3.7       1
17.7      1
15.7      1
6.7       1
4.7       1
19.7      1
Name: engine_displacement_l, dtype: int64

Много неправды. У Шевроле Нивы только один реальный объём: $1.7$ литра.\
Проверено **[здесь](https://www.drom.ru/catalog/chevrolet/niva/g_2009_1658/).** \
Нужно все аномалии заменить здесь модой.

Все остальные транспорты с очень большим объёмом — это или **большие автобусы**, или **крупнотоннажные грузовики**. Что выглядит правдоподобно.

<br>

Итак, заменим показатели мощности двигателя у электрокаров на пропуски.

In [106]:
# замена на пропуски
df.loc[df['fuel_type'] == 'Электричество', 'engine_displacement_l'] = np.nan

Теперь заменим все значения объёмов двигателя у Нив на модальное (реальное) значение. 

In [107]:
# замена модой
df.loc[df['model'] == 'Niva', 'engine_displacement_l'] = df[df['model'] == 'Niva']['engine_displacement_l'].mode().loc[0]

Посмотрим распределение нив снова.

In [108]:
df[df['model'] == 'Niva']['engine_displacement_l'].value_counts()

1.7    567
Name: engine_displacement_l, dtype: int64

Отлично. Всё исправлено.

<br>

Посмотрим колонку `engine_displacement_l` ещё раз.

In [109]:
duplicates('engine_displacement_l')

1.6    9075
2.0    8133
2.7    5074
2.5    5034
NaN    4275
1.5    1559
2.4    1247
1.7     833
3.5     788
4.6     754
Name: engine_displacement_l, dtype: int64

Количество непропущенных значений: 35691
Количество уникальных значений: 44

Первые 50 уникальных значений в алфавитном порядке:
[ 1.2  1.3  1.4  1.5  1.6  1.7  1.8  2.   2.2  2.4  2.5  2.7  2.8  2.9
  3.   3.3  3.5  3.6  3.8  3.9  4.   4.3  4.4  4.6  4.9  5.   5.2  5.5
  5.6  5.7  6.2  6.5  6.6  6.7  7.5  7.6  8.4  8.7 10.5 11.  12.  12.3
 12.8 13.   nan]


Количество уникальных значений снизилось с $80$ до $44$. Теперь данные стали значительно чище.

<br>

##### `Коробка передач`

In [110]:
duplicates('transmission_type')

AT                11379
6 AT               8842
MT                 3297
NaN                3255
CVT                2100
Мех.               2044
CVT (вариатор)     1462
6AT                1166
5МТ                 996
5 МТ                618
Name: transmission_type, dtype: int64

Количество непропущенных значений: 36711
Количество уникальных значений: 83

Первые 50 уникальных значений в алфавитном порядке:
[' 7 АКП' '#Н/Д' '0' '12AT' '12АТ' '4 АТ' '4AT' '4WD' '4АТ' '5 AT' '5 МТ'
 '5AT' '5M' '5MT' '5АТ' '5М' '5МТ' '6 AT' '6 DSG' '6 MT' '6 АТ' '6 МТ'
 '6A' '6AT' '6DSG' '6M/T' '6MT' '6А' '6АТ' '6МТ' '7 DCT' '7 DSG'
 '7 АКПП (PDK)' '7 АКПП PDK' '7AT' '7DCT' '7DSG' '7G-TRONIC' '7АКП' '7АТ'
 '8' '8 AT' '8 АКПП (PDK)' '8 АКПП Tiptronic S' '8 АТ' '8-ступ АКПП' '8A'
 '8AT' '8АКПП' '8АТ']


Полный хаос. $83$ коробок передач.\
Посмотрим их распределение.

In [111]:
# просмотр всего распределения
pd.set_option('display.max_rows', 100)

# распределение
print(df['transmission_type'].value_counts(dropna=False))

# вернуть кол-во строк по умолчанию
pd.reset_option('display.max_rows')

AT                    11379
6 AT                   8842
MT                     3297
NaN                    3255
CVT                    2100
Мех.                   2044
CVT (вариатор)         1462
6AT                    1166
5МТ                     996
5 МТ                    618
5MT                     614
AMT                     397
6MT                     384
6АТ                     383
6 АТ                    351
АКП                     329
АКПП                    230
CVT(вAриATор)           203
6 MT                    197
8AT                     151
4АТ                     133
АТ                      117
4 АТ                    105
МТ                      104
CVT (вариATор)           92
7DSG                     90
9G-TRONIC                72
6DSG                     71
АT                       61
DSG                      47
8АТ                      46
7 DSG                    42
МКП                      35
МКПП                     34
8 АТ                     34
Tiptronic           

<br>

У автомобилей с двигателями внутреннего сгорания есть всего $4$ коробки переключения передач:
1. Механическая (**MT** — Manual Transmission)
2. Автоматическая (**АТ** — Automatic Transmission)
3. Роботизированная (**АМТ** — Automated Manual Transmission)
4. Вариаторная (**CVT** — Continuously Variable Transmission)

У электрокаров роль трансмиссии выполняет **редуктор**.

Поэтому все $83$ значения колонки нужно привести к этим $5$.

<br>

Сначала можно убрать пробелы и цифры из значений.

In [112]:
df['transmission_type'] = df['transmission_type'].str.replace(' ', '')\
                                                 .str.replace('[0-9]', '', regex=True)

<br>

Посмотрим, что поменялось.

In [113]:
duplicates('transmission_type')

AT               21586
MT                4492
NaN               3255
CVT               2100
Мех.              2044
МТ                1747
CVT(вариатор)     1462
АТ                1182
AMT                397
АКП                338
Name: transmission_type, dtype: int64

Количество непропущенных значений: 36711
Количество уникальных значений: 43

Первые 50 уникальных значений в алфавитном порядке:
['' '#Н/Д' '-ступАКПП' 'A' 'A/T' 'AMT' 'AT' 'CVT' 'CVT(вAриATор)'
 'CVT(вариATор)' 'CVT(вариатор)' 'DCT' 'DSG' 'G-TRONIC' 'M' 'M/T' 'MT'
 'PDK' 'PowershiftS' 'S-Tronic' 'S-tronic' 'Steptronic' 'TDI' 'Tiptronic'
 'WD' 'А' 'А/T' 'АT' 'АКП' 'АКПП' 'АКПП(PDK)' 'АКППPDK' 'АКППTiptronicS'
 'АТ' 'М' 'М/T' 'МT' 'МКП' 'МКПП' 'МТ' 'Мех.' 'Передний' 'РЕДУКТОР' nan]


Значений стало почти в $2$ раза меньше.\
Нужно устранить неявные дубли.

<br>

Преобразуем значения так, чтобы они стали более понятны простому человеку.

Создадим списки, в которых будут дубли определённых коробок передач.

In [114]:
# механическая
mt = ['M', 'M/T', 'MT', 'М', 'М/T', 'МT', 'МКП', 'МКПП', 'МТ', 'Мех.']

# автоматическая
at = ['А', 'А/T', 'АT', 'АКП', 'АКПП', 'АТ', 'G-TRONIC', 'AT', 'A', 'A/T', 'Tiptronic', 'Steptronic', 'АКППTiptronicS', '-ступАКПП']

# робот
amt = ['PDK', 'PowershiftS', 'S-Tronic', 'АКПП(PDK)', 'АКППPDK', 'DCT', 'DSG', 'AMT', 'S-tronic']

# вариатор
cvt = ['CVT', 'CVT(вAриATор)', 'CVT(вариATор)', 'CVT(вариатор)']

# пропуски
nan = ['TDI', 'Передний', '#Н/Д', 'WD', '']

Создадим специальную конструкцию, которая поможет заменить значения.

In [115]:
# каждому значению (ключу) из созданных выше переменных назначено новое одно из пяти значений (значение) 
transmission_mapping = {
    **{value: 'MT' for value in mt},
    **{value: 'AT' for value in at},
    **{value: 'AMT' for value in amt},
    **{value: 'CVT' for value in cvt},
    **{value: np.NaN for value in nan}
}
# методу replace передаём словарь, в котором ключи — значения, которые нужно заменить, а значения — соответствующие новые значения
df['transmission_type'] = df['transmission_type'].replace(transmission_mapping)\
                                                 .replace('РЕДУКТОР', 'Редуктор')

Поменяем неточности в коробках передач там, где имеется возможность.

In [116]:
# cayenne
df.loc[df['model'] == 'Cayenne', 'transmission_type'] = 'AT'
df.loc[df['fuel_type'] == 'Электричество', 'transmission_type'] = 'Редуктор'

<br>

Посмотрим финальный результат.

In [117]:
duplicates('transmission_type')

AT          23637
MT           8409
CVT          3857
NaN          3305
AMT           745
Редуктор       13
Name: transmission_type, dtype: int64

Количество непропущенных значений: 36661
Количество уникальных значений: 5

Первые 50 уникальных значений в алфавитном порядке:
['AMT' 'AT' 'CVT' 'MT' 'Редуктор' nan]


Теперь в колонке наведён порядок.

<br>

##### `Тип привода`

In [118]:
duplicates('drive_type')

4WD         9412
2 WD        5078
2WD         4678
FF          4578
NaN         4289
Передний    3270
4 WD        2607
Полный      1917
Задний      1469
FWD          995
Name: drive_type, dtype: int64

Количество непропущенных значений: 35677
Количество уникальных значений: 26

Первые 50 уникальных значений в алфавитном порядке:
['#Н/Д' '0' '2 WD' '2WD' '4 WD' '4Motion' '4WD' '4X2' '4X4' '4x2' '4x4'
 '4х2' '4х2.2' '4х4' 'AWD' 'FF' 'FWD' 'RWD' 'quattro' 'Астана' 'Задний'
 'Передний' 'Передний (FF)' 'Полный' 'передний' 'полный' nan]


Здесь тоже нужно привести всё в порядок.

<br>

Рассмотрим распределение всех значений в колонке.

In [119]:
df['drive_type'].value_counts(dropna=False)

4WD              9412
2 WD             5078
2WD              4678
FF               4578
NaN              4289
Передний         3270
4 WD             2607
Полный           1917
Задний           1469
FWD               995
AWD               962
RWD               205
Передний (FF)     136
0                 116
передний           56
quattro            47
4х2                38
4Motion            26
4x4                22
4х4                18
4x2                16
4х2.2              10
4X4                 6
полный              5
4X2                 4
#Н/Д                4
Астана              2
Name: drive_type, dtype: int64

<br>

Вообще всего существует $4$ вида приводов:
1. Передний (**FWD** — Front Wheel Drive)
2. Задний (**RWD** — Rear Wheel Drive)
3. Полный (**4WD** — Four Wheel Drive)
4. Полный (**AWD** — All Wheel Drive)

<br>

В соответствии с этими приводами нужно привести все значения в колонке.

Создадим списки со значениями, которые нужно заменить на нормальные.

In [120]:
# двухколёсный (2WD)
twd = ['2 WD', '2WD', '4X2', '4x2', '4х2', '4х2.2']

# передний (FWD)
fwd = ['FF', 'FWD', 'Передний', 'Передний (FF)', 'передний']

# задний (RWD)
rwd = ['RWD', 'Задний']

# полный (4WD)
fourwd = ['4WD', '4 WD', '4Motion', '4X4', '4x4', '4х4', 'quattro', 'Полный', 'полный']

# полный (AWD)
awd = ['AWD']

# пропуски
nan = ['#Н/Д', '0', 'Астана']

Создадим конструкцию, которая заменит все неявные дубликаты.

In [121]:
# словарь, ключи которого содержат необработанные значения. Эти необработанные значения будут заменены валидными значенями
drive_type_mapping = {
    **{value: '2WD' for value in twd},
    **{value: 'FWD' for value in fwd},
    **{value: 'RWD' for value in rwd},
    **{value: '4WD' for value in fourwd},
    **{value: 'AWD' for value in awd},
    **{value: np.NaN for value in nan}
}

# замена значений
df['drive_type'] = df['drive_type'].replace(drive_type_mapping)

<br>

Теперь проверим работу.

In [122]:
duplicates('drive_type')

4WD    14060
2WD     9824
FWD     9035
NaN     4411
RWD     1674
AWD      962
Name: drive_type, dtype: int64

Количество непропущенных значений: 35555
Количество уникальных значений: 5

Первые 50 уникальных значений в алфавитном порядке:
['2WD' '4WD' 'AWD' 'FWD' 'RWD' nan]


Всё замечательно. Идём дальше.

<br>

`Области Казахстана`

In [123]:
duplicates('region')

г.Алматы                          11179
г.Нур-Султан                       8244
Южно-Казахстанская область         2880
Костанайская область               2494
Атырауская область                 2164
Карагандинская область             2076
Экспорт область                    1810
Мангистауская область              1495
Западно-Казахстанская область      1493
Восточно-Казахстанская область     1453
Name: region, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 18

Первые 50 уникальных значений в алфавитном порядке:
['Акмолинская область' 'Актюбинская область' 'Алматинская область'
 'Атырауская область' 'Восточно-Казахстанская область'
 'Жамбылская область' 'Западно-Казахстанская область'
 'Карагандинская область' 'Костанайская область' 'Кызылординская область'
 'Мангистауская область' 'Павлодарская область'
 'Северо-Казахстанская область' 'Туркестанская область' 'Экспорт область'
 'Южно-Казахстанская область' 'г.Алматы' 'г.Нур-Султан']


Колонка в полном порядке.

<br>

`Города Казахстана`

In [124]:
duplicates('city')

Алматы        11179
Нур-Султан     8244
Шымкент        2880
Костанай       2447
Атырау         2162
Караганда      2076
ЭКСПОРТ        1810
Актау          1495
Уральск        1493
Актобе         1351
Name: city, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 26

Первые 50 уникальных значений в алфавитном порядке:
['Актау' 'Актобе' 'Алматы' 'Атырау' 'Зыряновск' 'Караганда' 'Каскелен'
 'Кокшетау' 'Костанай' 'Кульсары' 'Кызылорда' 'Нур-Султан' 'Павлодар'
 'Петропавловск' 'Риддер' 'Рудный' 'Сарыагаш' 'Семей' 'Талдыкорган'
 'Тараз' 'Туркестан' 'Уральск' 'Усть-Каменогорск' 'Шымкент' 'ЭКСПОРТ'
 'Экибастуз']


Колонка в полном порядке.

<br>

##### `Наименование дилерского центра`

In [125]:
duplicates('dealer_center_name')

Hyundai Premium Almaty    1600
Hyundai Auto Almaty       1425
Hyundai Premium Astana    1385
Тойота Центр Алматы       1343
Тойота Центр Астана       1333
Toyota City Астана        1319
Toyota City               1258
Тойота Центр Жетысу       1150
HYUNDAI AUTO ASTANA       1117
Allur Auto Almaty         1079
Name: dealer_center_name, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 205

Первые 50 уникальных значений в алфавитном порядке:
['ABS-INVEST' 'ALAN MOTORS' 'ALAN MOTORS\xa0Усть-Каменогорск'
 'AST-commerce' 'ASTER AUTO CЕМЕЙ' 'ASTER AUTO Алматы'
 'ASTER AUTO Шымкент' 'Aktobe Hyundai Motors' 'Alan Motors Trade'
 'Allur Auto Almaty' 'Allur Auto Astana' 'Allur Auto Atyrau '
 'Allur Auto Karaganda' 'Allur Auto Kokshetau' 'Allur Auto Pavlodar '
 'Allur Auto Petropavlovsk ' 'Allur Auto Rus' 'Allur Auto Shymkent '
 'Allur Auto Taraz' 'Allur Auto Ust-Kamenogorsk ' 'Allur Auto Актобе'
 'Allur Auto Костанай' 'Allur Auto Кызылорда' 'Almaty Motors Premi

Здесь всё кажется в порядке. Вижу лишь несколько шероховатостей:
1. Убрать пробелы в конце названий у некоторых центров, например, в `'Allur Auto Petropavlovsk '`.
2. Убрать последствия кодировки `\xa0` в `'ALAN MOTORS\xa0Усть-Каменогорск'`.
3. Убрать $1$ дубль: `'Auto trader'` >> `'Auto Trader'`.

Исправим всё это.

In [126]:
# уберём пробелы
df['dealer_center_name'] = df['dealer_center_name'].str.strip(' ')

# уберём \xa0
df['dealer_center_name'] = df['dealer_center_name'].str.replace('\xa0', ' ')

# уберём  1 дублик
df['dealer_center_name'] = df['dealer_center_name'].replace('Auto trader', 'Auto Trader')

Как по мне, теперь колонка в полном порядке.

<br>

##### `Тип покупателя`

In [127]:
duplicates('customer_type')

Физ. Лицо       24568
Юр. Лицо         7873
NaN              7047
Корп. клиент      161
ФизЛицо           132
юридическое        97
ЮрЛицо             30
физ.лицо           29
Юр.Лицо            25
физическое          4
Name: customer_type, dtype: int64

Количество непропущенных значений: 32919
Количество уникальных значений: 9

Первые 50 уникальных значений в алфавитном порядке:
['Корп. клиент' 'Физ. Лицо' 'ФизЛицо' 'Юр. Лицо' 'Юр.Лицо' 'ЮрЛицо'
 'физ.лицо' 'физическое' 'юридическое' nan]


Объединим всех в $2$ типа покупателей:
* Физическое лицо
* Юридическое лицо

В соответствии с этими $2$ типами и нужно привести все значения.

In [128]:
df['customer_type'] = df['customer_type'].replace('Физ. Лицо', 'Физическое лицо')\
                                         .replace('ФизЛицо', 'Физическое лицо')\
                                         .replace('физ.лицо', 'Физическое лицо')\
                                         .replace('физическое', 'Физическое лицо')\
                                         .replace('Юр. Лицо', 'Юридическое лицо')\
                                         .replace('Юр.Лицо', 'Юридическое лицо')\
                                         .replace('ЮрЛицо', 'Юридическое лицо')\
                                         .replace('юридическое', 'Юридическое лицо')\
                                         .replace('Корп. клиент', 'Юридическое лицо')

Проверим.

In [129]:
duplicates('customer_type')

Физическое лицо     24733
Юридическое лицо     8186
NaN                  7047
Name: customer_type, dtype: int64

Количество непропущенных значений: 32919
Количество уникальных значений: 2

Первые 50 уникальных значений в алфавитном порядке:
['Физическое лицо' 'Юридическое лицо' nan]


Всё в порядке.

<br>

##### `Форма оплаты`

In [130]:
duplicates('payment_form')

NaN                                         25928
кредит                                       6190
безналичный                                  4276
Безналичная                                  1335
Собственные средства                          698
наличный                                      599
Наличная                                      499
Безналичная                                   204
Акция Собственные средства (Trade in 5%)       78
нал                                            58
Name: payment_form, dtype: int64

Количество непропущенных значений: 14038
Количество уникальных значений: 21

Первые 50 уникальных значений в алфавитном порядке:
['trade-in' 'Акция Собственные средства (Trade in 5%)' 'Без оплаты'
 'Без.Наличный' 'Безналичная' 'Безналичная ' 'Наличная' 'Отсрочка платежа'
 'Перечисление' 'Собственные средства' 'б/н' 'безналичный' 'безналичный '
 'кредит' 'лизинг' 'нал' 'наличные' 'наличный' 'обмен+кредит' 'обмен+нал'
 'перечислением' nan]


<br>

Глянем распределение форм платежей.

In [131]:
df['payment_form'].value_counts(dropna=False)

NaN                                         25928
кредит                                       6190
безналичный                                  4276
Безналичная                                  1335
Собственные средства                          698
наличный                                      599
Наличная                                      499
Безналичная                                   204
Акция Собственные средства (Trade in 5%)       78
нал                                            58
trade-in                                       35
Перечисление                                   21
Без оплаты                                      8
б/н                                             8
перечислением                                   6
Отсрочка платежа                                6
наличные                                        6
Без.Наличный                                    5
лизинг                                          2
безналичный                                     2


Просто выделим $2$ категории:
* В кредит
* Своими деньгами

<br>

Не впервые решая эту задачу, создадим списки.

In [132]:
# в кредит
credit = ['кредит', 'Отсрочка платежа', 'лизинг', 'обмен+кредит', 'trade-in']

# наличный расчёт
cash = ['Без.Наличный', 'Безналичная', 'Безналичная ', 'Перечисление', 'б/н', 'безналичный', 'безналичный ', 'перечислением',\
        'обмен+нал', 'нал', 'наличные', 'наличный', 'Наличная', 'Акция Собственные средства (Trade in 5%)', 'Собственные средства']

# без оплаты
other = ['Без оплаты']

Заменим значения, устранив при этом дубликаты.

In [133]:
# словарь с заменами
payment_mapping = {
    **{value: 'В кредит' for value in credit},
    **{value: 'Наличный расчёт' for value in cash},
    **{value: np.NaN for value in other}
}

# переиначивание
df['payment_form'] = df['payment_form'].replace(payment_mapping)

<br>

Проверим.

In [134]:
duplicates('payment_form')

NaN                25936
Наличный расчёт     7796
В кредит            6234
Name: payment_form, dtype: int64

Количество непропущенных значений: 14030
Количество уникальных значений: 2

Первые 50 уникальных значений в алфавитном порядке:
['В кредит' 'Наличный расчёт' nan]


Разобрались.

<br>

##### `Количество авто в заказе`

In [135]:
duplicates('quantity')

1     37203
2      1017
3       452
4       282
5       185
6       120
7       100
8        79
10       63
9        61
Name: quantity, dtype: Int64

Количество непропущенных значений: 39960
Количество уникальных значений: 56

Первые 50 уникальных значений в алфавитном порядке:
<IntegerArray>
[-1,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
 38, 40, 41, 42, 43, 46, 47, 50, 51, 54, 60, 62]
Length: 50, dtype: Int64


Среди всех значений выбивается только одно: `-1`. Его нужно заменить просто на `1`.\
Пусть не смутит большое количество купленых автомобилей за раз.\
Потому что если "в заказе" больше двух автомобилей, то это явно юридическое лицо, и оно покупает авто с коммерческими целями.

In [136]:
# замена -1 на 1
df['quantity'] = df['quantity'].replace(-1, 1)

Поехали дальше.

<br>

`Цена авто в $`

In [137]:
duplicates('price_usd')

9885     559
11600    531
13800    428
12820    412
9894     402
7100     389
9969     315
11300    230
32753    222
9000     219
Name: price_usd, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 3382

Первые 50 уникальных значений в алфавитном порядке:
[ 7100  7300  7363  7426  7460  7500  7509  7542  7560  7904  8027  8348
  8405  8498  8500  8684  8700  8781  8803  8809  8962  9000  9048  9112
  9117  9159  9260  9307  9455  9500  9519  9600  9612  9700  9773  9885
  9894  9911  9928  9969 10015 10068 10071 10080 10115 10117 10119 10121
 10122 10223]


Здесь нечего смотреть.

<br>

##### `Цена заказа в $`

In [138]:
duplicates('sale_usd')

9885     559
12820    412
9894     402
9969     315
13800    230
32753    222
41664    214
33342    210
10015    208
31291    181
Name: sale_usd, dtype: int64

Количество непропущенных значений: 39966
Количество уникальных значений: 4054

Первые 50 уникальных значений в алфавитном порядке:
[-35588      0   7100   7300   7363   7426   7460   7500   7509   7542
   7560   7904   8027   8348   8405   8498   8500   8684   8700   8781
   8803   8809   8962   9000   9048   9112   9117   9159   9260   9307
   9455   9500   9519   9600   9612   9700   9773   9885   9894   9911
   9928   9969  10015  10068  10071  10080  10115  10117  10119  10121]


<br>

Из этого списка выбиваются следующие значения.

In [139]:
df[(df['sale_usd'] == 0) | (df['sale_usd'] == -35588)]

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
31874,31874,2019-08-01,БИПЭК АВТО,Внедорожники,Kia,Sportage,Компактные SUV,2019,Локальное производство,Казахстан,Бензин,2.0,AT,4WD,Карагандинская область,Караганда,БИПЭК АВТО Караганда,NaN,NaN,<NA>,20700,0
31900,31900,2019-08-01,БИПЭК АВТО,Легковые автомобили,Lada,Granta,B класс,2019,Локальное производство,Казахстан,Бензин,1.6,AMT,FWD,г.Алматы,Алматы,БИПЭК АВТО Алматы,NaN,NaN,<NA>,7100,0
32068,32068,2019-08-01,БИПЭК АВТО,Легковые автомобили,Lada,XRAY,B класс,2018,Локальное производство,Казахстан,Бензин,1.6,MT,FWD,Павлодарская область,Экибастуз,БИПЭК АВТО Экибастуз,NaN,NaN,<NA>,13800,0
33150,33150,2019-08-01,БИПЭК АВТО,Внедорожники,Skoda,Kodiaq,Полноразмерные SUV,2019,Локальное производство,Казахстан,Бензин,2.0,AMT,4WD,г.Нур-Султан,Нур-Султан,БИПЭК АВТО Астана,NaN,NaN,<NA>,40000,0
33177,33177,2019-08-01,БИПЭК АВТО,Легковые автомобили,Skoda,Superb,D класс,2019,Локальное производство,Казахстан,Бензин,2.0,AMT,4WD,Жамбылская область,Тараз,БИПЭК АВТО Тараз,NaN,NaN,1,35588,-35588
34568,34568,2019-08-01,БИПЭК АВТО,Коммерческие автомобили,UAZ,3909,Развозные автомобили,2018,Локальное производство,Казахстан,Бензин,2.7,MT,4WD,г.Алматы,Алматы,БИПЭК АВТО Алматы,NaN,NaN,<NA>,11200,0
34593,34593,2019-08-01,БИПЭК АВТО,Внедорожники,UAZ,Patriot,Среднеразмерные SUV,2018,Локальное производство,Казахстан,Бензин,2.7,MT,4WD,Восточно-Казахстанская область,Семей,БИПЭК АВТО Семей,NaN,NaN,<NA>,13800,0


Думаю, стоит просто присвоить им значения из колонки `price_usd`.

In [140]:
df.loc[(df['sale_usd'] == 0) | (df['sale_usd'] == -35588), 'sale_usd'] = df[(df['sale_usd'] == 0) | (df['sale_usd'] == -35588)]['price_usd']

Вероятно, так случилось, потому что в `quantity` есть пропуски.\
Но с этой колонкой закончили.

<br>

**С неявными дубликатами и аномальными значениями разобрались.**

<br>

### Пропущенные данные

Разберёмся подробнее с пропусками.

Выведем количество пропусков и их процент.

In [141]:
# количество пропусков
df.isna()\
  .sum()\
  .sort_values(ascending=False)

payment_form             25936
customer_type             7047
drive_type                4411
engine_displacement_l     4275
transmission_type         3305
fuel_type                 3140
manufacturing_year         501
quantity                     6
price_usd                    0
dealer_center_name           0
city                         0
region                       0
id                           0
purchase_date                0
manufacturing_country        0
assembly_type                0
car_classification           0
model                        0
brand                        0
car_type                     0
company                      0
sale_usd                     0
dtype: int64

In [142]:
# процент пропусков
round(df.isna()\
        .mean() * 100, 1)\
        .sort_values(ascending=False)

payment_form             64.9
customer_type            17.6
drive_type               11.0
engine_displacement_l    10.7
transmission_type         8.3
fuel_type                 7.9
manufacturing_year        1.3
price_usd                 0.0
quantity                  0.0
dealer_center_name        0.0
city                      0.0
region                    0.0
id                        0.0
purchase_date             0.0
manufacturing_country     0.0
assembly_type             0.0
car_classification        0.0
model                     0.0
brand                     0.0
car_type                  0.0
company                   0.0
sale_usd                  0.0
dtype: float64

Много пропусков в колонке `payment_form`, $64.9$%, когда-то раньше мы это отмечали.

Вот список колонок, где есть пропуски.

In [143]:
df.columns[df.isna().any()]

Index(['manufacturing_year', 'fuel_type', 'engine_displacement_l',
       'transmission_type', 'drive_type', 'customer_type', 'payment_form',
       'quantity'],
      dtype='object')

<br>

**Сделаем функцию, которая будет помогать анализировать пропуски.**

In [144]:
def null_analysis(df, column):
    # копия главного датафрейма
    nan_df = df.copy()
    
    # приведение к строковому типу данных, это нужно, чтобы пропуски записались как отдельное значение
    nan_df[column] = nan_df[column].astype('str')

    # сводная таблица
    nan_df = nan_df.pivot_table(index=['brand', 'model'], columns=column, aggfunc='size', fill_value=0).reset_index()

    # переименование колонки
    try:
        nan_df = nan_df.rename(columns={'nan': 'null_cnt'})
    except:
        pass
    
    try:
        nan_df = nan_df.rename(columns={'<NA>': 'null_cnt'})
    except:
        pass
    
    # выбор строк, где есть пропуски; сортировка по колонкам
    nan_df = nan_df[nan_df['null_cnt'] != 0].sort_values(by=['brand', 'null_cnt'], ascending=[True, False])
    
    # переместить 'null_cnt' в конец сводной таблицы: циклом берутся все колонки, кроме 'null_cnt', а затем к ним конкатенируется 'null_cnt'
    nan_df = nan_df[[col for col in nan_df.columns if col != 'null_cnt'] + ['null_cnt']]

    # переименование индекса
    nan_df = nan_df.rename_axis(columns='index')

    # преобразование числовых колонок в числовой тип
    nan_df.iloc[:, 2:] = nan_df.iloc[:, 2:].astype('Int64')

    # количество наблюдений по модели
    nan_df['obs_cnt'] = nan_df.iloc[:, 2:].sum(axis=1).astype('Int64')

    # процент пропусков по модели
    nan_df['null_perc'] = nan_df['null_cnt'] / nan_df['obs_cnt'] * 100

    # преобразование колонки с процентом
    nan_df['null_perc'] = round(nan_df['null_perc'].astype(float), 1)

    # какой процент занимает модальное значение от всех непропущенных значений; метод apply применяет lambda функцию к каждой строке (axis=1)
    nan_df['mode_perc'] = nan_df.apply(lambda row: round(row.iloc[2:-3].max() / (row['obs_cnt'] - row['null_cnt']) * 100, 1) if (row['obs_cnt'] - row['null_cnt']) != 0 and row.iloc[2:-3].max() != 0 else 0, axis=1)

    # сверка пропусков и наблюдений сводной таблицы с данными датафрейма
    result_str = ''
    if nan_df['null_cnt'].sum() == df[column].isna().sum():
        result_str += f'Количество пропущенных значений в сводной таблице и основном датафрейме сходится.\n'
        result_str += f'В сводной: {nan_df["null_cnt"].sum()} пропусков. В датафрейме: {df[column].isna().sum()} пропусков.\n\n'
    else:
        result_str += 'Количество пропущенных значений в сводной таблице и основном датафрейме не сходится.\n'
        result_str += f'В сводной: {nan_df["null_cnt"].sum()} пропусков. В датафрейме: {df[column].isna().sum()} пропусков.\n\n'

    if nan_df['obs_cnt'].sum() == len(df[df['model'].isin(df[df[column].isna()]['model'].unique())]):
        result_str += 'Количество наблюдений в сводной таблице и основном датафрейме сходится.\n'
        result_str += f'В сводной: {nan_df["obs_cnt"].sum()} наблюдений. В датафрейме: {len(df[df["model"].isin(df[df[column].isna()]["model"].unique())])} наблюдений.\n\n'
    else: 
        result_str += 'Количество наблюдений в сводной таблице и основном датафрейме не сходится.\n'
        result_str += f'В сводной: {nan_df["obs_cnt"].sum()} наблюдений. В датафрейме: {len(df[df["model"].isin(df[df[column].isna()]["model"].unique())])} наблюдений.\n\n'
    
    return nan_df, result_str

<br>

##### `Год производства`

In [145]:
df[df['manufacturing_year'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
5048,5048,2019-02-01,Автомир-Центр,Легковые автомобили,Mazda,3,C класс,<NA>,Импорт,Россия,NaN,NaN,NaN,NaN,Карагандинская область,Караганда,Автомир-Центр Караганда,NaN,Наличный расчёт,1,21547,21547
5049,5049,2019-02-01,Автомир-Центр,Легковые автомобили,Mazda,3,C класс,<NA>,Импорт,Россия,NaN,NaN,NaN,NaN,г.Нур-Султан,Нур-Султан,Автомир-Центр Астана,NaN,Наличный расчёт,1,21547,21547
5050,5050,2019-02-01,Автомир-Центр,Легковые автомобили,Mazda,6,D класс,<NA>,Импорт,Россия,NaN,NaN,NaN,NaN,г.Нур-Султан,Нур-Султан,Автомир-Центр Астана,NaN,В кредит,1,24998,24998
5051,5051,2019-02-01,Автомир-Центр,Внедорожники,Mazda,CX-5,Компактные SUV,<NA>,Импорт,Россия,NaN,NaN,NaN,NaN,Карагандинская область,Караганда,Автомир-Центр Караганда,NaN,В кредит,3,26684,80053
5052,5052,2019-02-01,Автомир-Центр,Внедорожники,Mazda,CX-5,Компактные SUV,<NA>,Импорт,Россия,NaN,NaN,NaN,NaN,Карагандинская область,Караганда,Автомир-Центр Караганда,NaN,Наличный расчёт,3,26684,80053


In [146]:
df['manufacturing_year'].value_counts(dropna=False).head(10)

2019    30026
2018     9081
NaN       501
2017      324
2016       30
2014        2
2011        1
2013        1
Name: manufacturing_year, dtype: Int64

Здесь пропуски — третье самое встречаемое значение.

<br>

Применим функцию.

In [147]:
pt_man, str_man = null_analysis(df, 'manufacturing_year')

In [148]:
# вывод строк с пропусками и наблюдениями
print(str_man)

# компактная таблица
pt_man.head()

# раскомментируйте переменную, чтобы увидеть всю сводную таблицу
# pt_man

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 501 пропусков. В датафрейме: 501 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 4978 наблюдений. В датафрейме: 4978 наблюдений.




index,brand,model,2011,2013,2014,2016,2017,2018,2019,null_cnt,obs_cnt,null_perc,mode_perc
16,BMW,7 серия,0,0,0,0,0,1,5,1,7,14.3,83.3
65,Hyundai Truck & Bus,H350,0,0,0,0,5,0,3,32,40,80.0,62.5
70,Hyundai Truck & Bus,HD 78,0,0,0,0,1,0,11,30,42,71.4,91.7
68,Hyundai Truck & Bus,HD 35,0,0,0,0,4,0,11,15,30,50.0,73.3
69,Hyundai Truck & Bus,HD 65,0,0,0,0,4,5,7,8,24,33.3,43.8


**Итак, какую сводную таблицу создала функция:**
1. В качестве индексов выступают марка и модель автомобиля.
2. В качестве колонок выступают года, когда был произведён автомобиль.
3. С помощью агрегирующей функции мы подсчитываем количество автомобилей, произведённых в определённый год.
4. `null_cnt` значит количество пропусков у конкретной модели за все годы.
5. `obs_cnt` значит количество наблюдений **(это включая пропуски)** у конкретной модели.
6. `null_perc` какой процент от **всех наблюдений** занимают пропуски.
7. `mode_perc` значит, какой процент составляет модальное (самое частотное) значение из всех **непропущенных значений**.

Сводные таблицы по умолчанию **отсортированы по марке авто в лексикографическом порядке и по убыванию количества пропущенных значений.**

<br>

Заполнять пропуски будем модой.\
Я уверен, что это неточный способ, ведь чем ниже процент, который занимает модальное значение, тем выше вероятность неточности.\
Лучший способ мне пока неизвестен.\
**Применим правило: если модальное значение занимает процент, равный или больше $60$, то это модальное значение заменит пропуск в годе производства для этой модели.**

In [149]:
# итерация по строкам таблицы pt_man с использованием метода iterrows(): index — строка, row — колонка
for index, row in pt_man.iterrows():
    # извлечение значения 'model' для текущей строки
    model = row['model']
    # извлечение значения 'mode_perc' для текущей строки
    mode_perc = row['mode_perc']
    
    # если процент модального значения >= 60 для данной модели
    if mode_perc >= 60:
        # то находим модальное значение года производства для неё
        mode_value = df[df['model'] == model]['manufacturing_year'].mode()
        
        # если mode_value не является пропуском
        if pd.notna(mode_value.iloc[0]):
            # то заполняем пропуски в годе производства модальным значением для данной модели
            df.loc[(df['model'] == model) & (df['manufacturing_year'].isna()), 'manufacturing_year'] = mode_value.iloc[0]

<br>

Проверим результат.

In [150]:
# запишем в иные переменные
pt_man_fill, str_man_fill = null_analysis(df, 'manufacturing_year')

print(str_man_fill)
pt_man_fill

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 27 пропусков. В датафрейме: 27 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 203 наблюдений. В датафрейме: 203 наблюдений.




index,brand,model,2011,2013,2014,2016,2017,2018,2019,null_cnt,obs_cnt,null_perc,mode_perc
69,Hyundai Truck & Bus,HD 65,0,0,0,0,4,5,7,8,24,33.3,43.8
66,Hyundai Truck & Bus,HD 160,0,0,0,0,0,0,0,1,1,100.0,0.0
146,Land Rover,Range Rover,0,0,0,0,2,41,41,1,85,1.2,48.8
165,Mazda,3,0,0,0,0,0,0,0,5,5,100.0,0.0
196,Mitsubishi,Pajero Sport,0,0,0,0,0,27,32,11,70,15.7,54.2
267,Volvo,XC60,0,0,0,0,0,8,9,1,18,5.6,52.9


Итак, остались пропуски по этим моделям. 

<br>

##### `Тип топлива`

In [151]:
df[df['fuel_type'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
97,97,2019-01-01,СемАЗ,Коммерческие автомобили,Dong Feng,BWC6665GA5,Средние автобусы,2019,Локальное производство,Казахстан,NaN,NaN,NaN,NaN,г.Нур-Султан,Нур-Султан,СемАЗ,NaN,NaN,29,26693,774091
99,99,2019-01-01,СемАЗ,Коммерческие автомобили,Foton,BJ1069VDJEA-F1,Малотоннажные грузовики,2018,Локальное производство,Казахстан,NaN,NaN,NaN,NaN,г.Алматы,Алматы,Меридиан АВТО Алматы,NaN,NaN,3,13619,40858
1056,1056,2019-01-01,Allur Auto,Коммерческие автомобили,Iveco,Daily,Малотоннажные грузовики,2017,Локальное производство,Казахстан,NaN,NaN,NaN,NaN,г.Алматы,Алматы,Allur Auto Almaty,Юридическое лицо,NaN,1,27750,27750
1068,1068,2019-01-01,Allur Auto,Внедорожники,Jac,S3,Субкомпактные SUV,2018,Локальное производство,Казахстан,NaN,NaN,NaN,2WD,Экспорт область,ЭКСПОРТ,Allur Auto Almaty,Юридическое лицо,NaN,1,12818,12818
1069,1069,2019-01-01,Allur Auto,Внедорожники,Jac,S3,Субкомпактные SUV,2018,Локальное производство,Казахстан,NaN,NaN,NaN,2WD,Экспорт область,ЭКСПОРТ,Allur Auto Almaty,Юридическое лицо,NaN,1,12818,12818


In [152]:
df['fuel_type'].value_counts(dropna=False).head(10)

Бензин           35000
NaN               3140
Дизель            1803
Электричество       13
Гибрид              10
Name: fuel_type, dtype: int64

Здесь пропуски — второе самое встречаемое значение.

<br>

Применим функцию.

In [153]:
pt_fuel, str_fuel = null_analysis(df, 'fuel_type')

In [154]:
# вывод строк с пропусками и наблюдениями
print(str_fuel)

# раскомментируйте код, чтобы увидеть всю сводную таблицу
# pd.set_option('display.max_rows', 100)
pt_fuel

# раскомментируйте код, чтобы снова сделать таблицу компактной
# pd.reset_option('display.max_rows')
pt_fuel

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 3140 пропусков. В датафрейме: 3140 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 7373 наблюдений. В датафрейме: 7373 наблюдений.




index,brand,model,Бензин,Гибрид,Дизель,Электричество,null_cnt,obs_cnt,null_perc,mode_perc
3,ANKAI,HFF6850G,0,0,167,0,73,240,30.4,100.0
0,ANKAI,HF-D105,0,0,0,0,30,30,100.0,0.0
40,Dong Feng,BWC6665GA5,0,0,0,0,10,10,100.0,0.0
43,Foton,BJ1069VDJEA-F1,0,0,0,0,19,19,100.0,0.0
44,Foton,BJ3253DMPKB-AD,0,0,0,0,9,9,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...
271,Урал,4320,0,0,0,0,7,7,100.0,0.0
274,Урал,5557,0,0,0,0,2,2,100.0,0.0
270,Урал,32551,0,0,0,0,1,1,100.0,0.0
272,Урал,4320-1951-40,0,0,0,0,1,1,100.0,0.0


Проанализировав эту сводную таблицу, можно заполнить пропуски в типе топлива модальным значением.\
Только автомобили марки `Mazda` и `Renault` придётся заполнить вручную, там пропусков много. И ещё по некоторым грузовикам и автобусам то же самое нужно проделать.

**Применим правило: если модальное значение занимает процент, равный или больше  $60$, то это модальное значение заменит пропуск в виде топлива для этой модели.**

In [155]:
# итерация по строкам таблицы pt_fuel с использованием метода iterrows(): index — строка, row — колонка
for index, row in pt_fuel.iterrows():
    # извлечение значения 'model' для текущей строки
    model = row['model']
    # извлечение значения 'mode_perc' для текущей строки
    mode_perc = row['mode_perc']
    
    # если процент модального значения >= 60 для данной модели
    if mode_perc >= 60:
        # то находим модальное значение вида топлива для неё
        mode_value = df[df['model'] == model]['fuel_type'].mode()
        
        # если mode_value не является пропуском
        if pd.notna(mode_value.iloc[0]):
            # то заполняем пропуски в виде топлива модальным значением для данной модели
            df.loc[(df['model'] == model) & df['fuel_type'].isna(), 'fuel_type'] = mode_value.iloc[0]

<br>

Ручная замена.

In [156]:
diesel = ['Master', 'Dokker', 'SX3258DR384', 'HF-D105', 'BJ1069VDJEA-F1']
petrol = ['Logan', 'Kaptur', 'Arkana', 'Koleos', '6', '3', 'CX-5', 'CX-9']

In [157]:
for model in diesel:
    df.loc[(df['model'] == model) & (df['fuel_type'].isna()), 'fuel_type'] = 'Дизель'
    
for model in petrol:
    df.loc[(df['model'] == model) & (df['fuel_type'].isna()), 'fuel_type'] = 'Бензин'

<br>

Проверим результат.

In [158]:
# запишем в иные переменные
pt_fuel_fill, str_fuel_fill = null_analysis(df, 'fuel_type')

print(str_fuel_fill)
pt_fuel_fill.sort_values(by='null_cnt', ascending=False)

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 47 пропусков. В датафрейме: 47 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 47 наблюдений. В датафрейме: 47 наблюдений.




index,brand,model,Бензин,Гибрид,Дизель,Электричество,null_cnt,obs_cnt,null_perc,mode_perc
40,Dong Feng,BWC6665GA5,0,0,0,0,10,10,100.0,0
44,Foton,BJ3253DMPKB-AD,0,0,0,0,9,9,100.0,0
271,Урал,4320,0,0,0,0,7,7,100.0,0
64,Hyundai Truck & Bus,EX8,0,0,0,0,6,6,100.0,0
87,Iveco,Tipper,0,0,0,0,4,4,100.0,0
84,Iveco,Daily A,0,0,0,0,2,2,100.0,0
86,Iveco,Stralis,0,0,0,0,2,2,100.0,0
274,Урал,5557,0,0,0,0,2,2,100.0,0
66,Hyundai Truck & Bus,HD 160,0,0,0,0,1,1,100.0,0
85,Iveco,Daily V,0,0,0,0,1,1,100.0,0


Что за транспортные средства остались?

In [159]:
df.loc[df['model'].isin(pt_fuel_fill['model'].unique())]['car_classification'].unique()

array(['Средние автобусы', 'Малотоннажные грузовики',
       'Крупнотоннажные грузовики', 'Микроавтобусы',
       'Развозные автомобили', 'Среднетоннажные грузовики'], dtype=object)

Как видим, это грузовики и автобусы. Отнесём их к дизелю.

In [160]:
df.loc[df['model'].isin(pt_fuel_fill['model'].unique()), 'fuel_type'] = 'Дизель'

Проверим пропуски.

In [161]:
df['fuel_type'].isna().sum()

0

Пропусков здесь больше нет.

<br>

<br>

##### `Объём двигателя`

In [162]:
df[df['engine_displacement_l'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
97,97,2019-01-01,СемАЗ,Коммерческие автомобили,Dong Feng,BWC6665GA5,Средние автобусы,2019,Локальное производство,Казахстан,Дизель,NaN,NaN,NaN,г.Нур-Султан,Нур-Султан,СемАЗ,NaN,NaN,29,26693,774091
99,99,2019-01-01,СемАЗ,Коммерческие автомобили,Foton,BJ1069VDJEA-F1,Малотоннажные грузовики,2018,Локальное производство,Казахстан,Дизель,NaN,NaN,NaN,г.Алматы,Алматы,Меридиан АВТО Алматы,NaN,NaN,3,13619,40858
1052,1052,2019-01-01,СВС-ТРАНС,Коммерческие автомобили,Isuzu,NMS85,Среднетоннажные грузовики,2018,Импорт,Россия,Дизель,NaN,NaN,4WD,г.Алматы,Алматы,CBC Алматы,NaN,NaN,1,44146,44146
1053,1053,2019-01-01,СВС-ТРАНС,Коммерческие автомобили,Isuzu,NPR,Малотоннажные грузовики,2018,Импорт,Россия,Дизель,NaN,NaN,2WD,г.Алматы,Алматы,CBC Алматы,NaN,NaN,1,37000,37000
1054,1054,2019-01-01,СВС-ТРАНС,Коммерческие автомобили,Isuzu,NPR,Малотоннажные грузовики,2018,Импорт,Россия,Дизель,NaN,NaN,2WD,г.Алматы,Алматы,CBC Алматы,NaN,NaN,1,37000,37000


In [163]:
df['engine_displacement_l'].value_counts(dropna=False).head(10)

1.6    9075
2.0    8133
2.7    5074
2.5    5034
NaN    4275
1.5    1559
2.4    1247
1.7     833
3.5     788
4.6     754
Name: engine_displacement_l, dtype: int64

Здесь пропуски — пятое самое встречаемое значение.

<br>

Применим функцию.

In [164]:
pt_engine, str_engine = null_analysis(df, 'engine_displacement_l')

In [165]:
# вывод строк с пропусками и наблюдениями
print(str_engine)

# раскомментируйте код, чтобы увидеть всю сводную таблицу
# pd.set_option('display.max_rows', 110)
pt_engine

# раскомментируйте код, чтобы снова сделать таблицу компактной
# pd.reset_option('display.max_rows')
pt_engine.sort_values(by='null_perc', ascending=False)

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 4275 пропусков. В датафрейме: 4275 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 11975 наблюдений. В датафрейме: 11975 наблюдений.




index,brand,model,1.2,1.3,1.4,1.5,1.6,1.7,1.8,10.5,11.0,12.0,12.3,12.8,13.0,2.0,2.2,2.4,2.5,2.7,2.8,2.9,3.0,3.3,3.5,3.6,3.8,3.9,4.0,4.3,4.4,4.6,4.9,5.0,5.2,5.5,5.6,5.7,6.2,6.5,6.6,6.7,7.5,7.6,8.4,8.7,null_cnt,obs_cnt,null_perc,mode_perc
118,KAMAZ,58815Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,15,100.0,0.0
116,KAMAZ,54115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,17,100.0,0.0
125,KAMAZ,65225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,100.0,0.0
110,KAMAZ,45141,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,100.0,0.0
124,KAMAZ,65206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Volkswagen,Polo,0,0,0,0,512,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,28,544,5.1,99.2
51,GAZ,3308,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,55,1.8,98.1
146,Land Rover,Range Rover,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0,29,0,0,0,0,0,0,0,0,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,1,85,1.2,46.4
237,Toyota,HILUX DC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,298,0,502,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,827,0.2,60.8


Здесь ситуация сложнее.

<br>

**Применим правило: если модальное значение занимает процент, равный или больше  $60$, и процент пропусков не превышает $90$, то это модальное значение заменит пропуск в виде топлива для этой модели.**

In [166]:
# итерация по строкам таблицы pt_fuel с использованием метода iterrows(): index — строка, row — колонка
for index, row in pt_engine.iterrows():
    # извлечение значения 'model' для текущей строки
    model = row['model']
    # извлечение значения 'mode_perc' для текущей строки
    mode_perc = row['mode_perc']
    # извлечение значения 'null_perc' для текущей строки
    null_perc = row['null_perc']
    
    # если процент модального значения >= 60 для данной модели и процент пропусков не больше 90
    if mode_perc >= 60 and null_perc <= 90:
        # то находим модальное значение вида топлива для неё
        mode_value = df[df['model'] == model]['engine_displacement_l'].mode()
        
        # если mode_value не является пустотой из-за того, что все значения у модели уникальны, and если mode_value не является пропуском
        if not mode_value.empty and pd.notna(mode_value.iloc[0]):
            # то заполняем пропуски в виде топлива модальным значением для данной модели
            df.loc[(df['model'] == model) & df['engine_displacement_l'].isna(), 'engine_displacement_l'] = mode_value.iloc[0]

<br>

По моделям, у которых вообще не указан объём двигателя, одни пропуски:\
`Logan` и `Sandero` есть только в одном виде — $1.6$ литра.\
Вообще, где буду указывать объём двигателя вручную, там я руководствовался информацией из одного автомобильного портала, где продаются машины.

По другим автомобилям `Renault` есть `2` мотора разных объёмов. Мы не можем здесь угадывать.\
По автомобилям `Mazda` та же картина. Тут не угадаешь.\
Единственное, что логически можно рассудить, так это `Duster`. Есть $2$ модели: стоимостью за $12$К, и за $15$К. Так и распределим объём.

<br>

Заполним двигатели транспорта, в которых я уверен.

In [167]:
# Renault
df.loc[(df['model'] == 'Logan') | (df['model'] == 'Sandero'), 'engine_displacement_l'] = 1.6
df.loc[(df['model'] == 'Duster') & (df['engine_displacement_l'].isna()) & (df['price_usd'] > 15000), 'engine_displacement_l'] = 2.0
df.loc[(df['model'] == 'Duster') & (df['engine_displacement_l'].isna()) & (df['price_usd'] < 13000), 'engine_displacement_l'] = 1.6

# КамАЗы
df.loc[df['model'].isin(['65115', '45143', '65116', '65117', '43253', '58815Z', '53605']), 'engine_displacement_l'] = 6.7
df.loc[df['model'].isin(['43118', '6520', '44108', '53215', '45142', '54115', '53504']), 'engine_displacement_l'] = 10.8
df.loc[df['model'].isin(['65111', '43502']), 'engine_displacement_l'] = 11.8
df.loc[df['model'].isin(['5490', '65206']), 'engine_displacement_l'] = 12

# другой транспорт
df.loc[df['model'] == 'HD 78', 'engine_displacement_l'] = 3.9
df.loc[df['model'] == 'SX3258DR384', 'engine_displacement_l'] = 9.7

<br>

Проверим результат.

In [168]:
# запишем в иные переменные
pt_engine_fill, str_engine_fill = null_analysis(df, 'engine_displacement_l')

print(str_engine_fill)
pt_engine_fill.sort_values(by='null_cnt', ascending=False).head()

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 594 пропусков. В датафрейме: 594 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 679 наблюдений. В датафрейме: 679 наблюдений.




index,brand,model,1.2,1.3,1.4,1.5,1.6,1.7,1.8,10.5,10.8,11.0,11.8,12.0,12.3,12.8,13.0,2.0,2.2,2.4,2.5,2.7,2.8,2.9,3.0,3.3,3.5,3.6,3.8,3.9,4.0,4.3,4.4,4.6,4.9,5.0,5.2,5.5,5.6,5.7,6.2,6.5,6.6,6.7,7.5,7.6,8.4,8.7,9.7,null_cnt,obs_cnt,null_perc,mode_perc
215,Renault,Kaptur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,230,230,100.0,0.0
212,Renault,Arkana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,55,100.0,0.0
167,Mazda,CX-5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,51,100.0,0.0
213,Renault,Dokker,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,34,100.0,0.0
0,ANKAI,HF-D105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,30,100.0,0.0


Остальные пропуски оставим в покое. Овчинка выделки не стоит.

<br>

##### `Коробка передач`

In [169]:
df[df['transmission_type'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
97,97,2019-01-01,СемАЗ,Коммерческие автомобили,Dong Feng,BWC6665GA5,Средние автобусы,2019,Локальное производство,Казахстан,Дизель,NaN,NaN,NaN,г.Нур-Султан,Нур-Султан,СемАЗ,NaN,NaN,29,26693,774091
99,99,2019-01-01,СемАЗ,Коммерческие автомобили,Foton,BJ1069VDJEA-F1,Малотоннажные грузовики,2018,Локальное производство,Казахстан,Дизель,NaN,NaN,NaN,г.Алматы,Алматы,Меридиан АВТО Алматы,NaN,NaN,3,13619,40858
1052,1052,2019-01-01,СВС-ТРАНС,Коммерческие автомобили,Isuzu,NMS85,Среднетоннажные грузовики,2018,Импорт,Россия,Дизель,NaN,NaN,4WD,г.Алматы,Алматы,CBC Алматы,NaN,NaN,1,44146,44146
1053,1053,2019-01-01,СВС-ТРАНС,Коммерческие автомобили,Isuzu,NPR,Малотоннажные грузовики,2018,Импорт,Россия,Дизель,5.2,NaN,2WD,г.Алматы,Алматы,CBC Алматы,NaN,NaN,1,37000,37000
1054,1054,2019-01-01,СВС-ТРАНС,Коммерческие автомобили,Isuzu,NPR,Малотоннажные грузовики,2018,Импорт,Россия,Дизель,5.2,NaN,2WD,г.Алматы,Алматы,CBC Алматы,NaN,NaN,1,37000,37000


In [170]:
df['transmission_type'].value_counts(dropna=False).head(10)

AT          23637
MT           8409
CVT          3857
NaN          3305
AMT           745
Редуктор       13
Name: transmission_type, dtype: int64

Здесь пропуски — четвёртое самое встречаемое значение.

<br>

Применим функцию.

In [171]:
pt_transm, str_transm = null_analysis(df, 'transmission_type')

In [172]:
# вывод строк с пропусками и наблюдениями
print(str_transm)

# раскомментируйте код, чтобы увидеть всю сводную таблицу
# pd.set_option('display.max_rows', 80)
pt_transm

# раскомментируйте код, чтобы снова сделать таблицу компактной
# pd.reset_option('display.max_rows')
pt_transm.sort_values(by=['mode_perc', 'null_perc'], ascending=[False, True])

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 3305 пропусков. В датафрейме: 3305 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме не сходится.
В сводной: 9307 наблюдений. В датафрейме: 9308 наблюдений.




index,brand,model,AMT,AT,CVT,MT,Редуктор,null_cnt,obs_cnt,null_perc,mode_perc
146,Land Rover,Range Rover,0,84,0,0,0,1,85,1.2,100.0
267,Volvo,XC60,0,17,0,0,0,1,18,5.6,100.0
149,Land Rover,Range Rover Velar,0,15,0,0,0,1,16,6.2,100.0
148,Land Rover,Range Rover Sport,0,14,0,0,0,1,15,6.7,100.0
268,Volvo,XC90,0,13,0,0,0,1,14,7.1,100.0
...,...,...,...,...,...,...,...,...,...,...,...
271,Урал,4320,0,0,0,0,0,7,7,100.0,0.0
274,Урал,5557,0,0,0,0,0,2,2,100.0,0.0
270,Урал,32551,0,0,0,0,0,1,1,100.0,0.0
272,Урал,4320-1951-40,0,0,0,0,0,1,1,100.0,0.0


<br>

**Применим правило: если модальное значение занимает процент, равный $100$, и процент пропусков не превышает $90$, то это модальное значение заменит пропуск в типе трансмиссии для этой модели.**

In [173]:
# итерация по строкам таблицы pt_transm с использованием метода iterrows(): index — строка, row — колонка
for index, row in pt_transm.iterrows():
    # извлечение значения 'model' для текущей строки
    model = row['model']
    # извлечение значения 'mode_perc' для текущей строки
    mode_perc = row['mode_perc']
    # извлечение значения 'null_perc' для текущей строки
    null_perc = row['null_perc']
    
    # если процент модального значения >= 100 для данной модели и процент пропусков не больше 90
    if mode_perc == 100 and null_perc <= 90:
        # то находим модальное значение вида топлива для неё
        mode_value = df[df['model'] == model]['transmission_type'].mode()
        
        # если mode_value не является пустотой из-за того, что все значения у модели уникальны, and если mode_value не является пропуском
        if not mode_value.empty and pd.notna(mode_value.iloc[0]):
            # то заполняем пропуски в виде топлива модальным значением для данной модели
            df.loc[(df['model'] == model) & df['transmission_type'].isna(), 'transmission_type'] = mode_value.iloc[0]

Проанализировав сводную таблицу, решил, что применённое правило неплохо ляжет на пропущенные значения.

<br>

Заполним типы трансмиссий транспорта, в которых я уверен.

In [174]:
# у Nexia R3 пропуски там, где самые дорогие комплектации, это АКПП
df.loc[(df['model'] == 'Nexia R3') & (df['transmission_type'].isna()), 'transmission_type'] = 'AT'

Дальнейшая замена пропусков кажется почти нереальной.

<br>

Проверим результат.

In [175]:
# запишем в иные переменные
pt_transm_fill, str_transm_fill = null_analysis(df, 'transmission_type')

print(str_transm_fill)
pt_transm_fill.sort_values(by=['null_cnt'], ascending=[False]).head()

# раскомментируйте код, чтобы увидеть весь результат
# pt_transm_fill.sort_values(by=['null_cnt'], ascending=[False])

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 2724 пропусков. В датафрейме: 2724 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 6822 наблюдений. В датафрейме: 6822 наблюдений.




index,brand,model,AMT,AT,CVT,MT,Редуктор,null_cnt,obs_cnt,null_perc,mode_perc
214,Renault,Duster,0,13,0,12,0,682,707,96.5,52.0
219,Renault,Sandero,0,0,0,1,0,440,441,99.8,100.0
217,Renault,Logan,0,0,0,0,0,320,320,100.0,0.0
215,Renault,Kaptur,0,0,0,0,0,230,230,100.0,0.0
91,Jac,S3,0,0,821,290,0,175,1286,13.6,73.9


Это всё. Дальнейшая замена пропусков кажется почти нереальной.

<br>

##### `Тип привода`

In [176]:
df[df['drive_type'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
97,97,2019-01-01,СемАЗ,Коммерческие автомобили,Dong Feng,BWC6665GA5,Средние автобусы,2019,Локальное производство,Казахстан,Дизель,NaN,NaN,NaN,г.Нур-Султан,Нур-Султан,СемАЗ,NaN,NaN,29,26693,774091
99,99,2019-01-01,СемАЗ,Коммерческие автомобили,Foton,BJ1069VDJEA-F1,Малотоннажные грузовики,2018,Локальное производство,Казахстан,Дизель,NaN,NaN,NaN,г.Алматы,Алматы,Меридиан АВТО Алматы,NaN,NaN,3,13619,40858
1056,1056,2019-01-01,Allur Auto,Коммерческие автомобили,Iveco,Daily,Малотоннажные грузовики,2017,Локальное производство,Казахстан,Дизель,NaN,MT,NaN,г.Алматы,Алматы,Allur Auto Almaty,Юридическое лицо,NaN,1,27750,27750
1129,1129,2019-01-01,ТК КАМАЗ,Коммерческие автомобили,KAMAZ,43118,Крупнотоннажные грузовики,2018,Локальное производство,Казахстан,Дизель,10.8,MT,NaN,г.Нур-Султан,Нур-Султан,Российские Грузовики Астана,Юридическое лицо,NaN,1,49950,49950
1130,1130,2019-01-01,ТК КАМАЗ,Коммерческие автомобили,KAMAZ,43118,Крупнотоннажные грузовики,2018,Локальное производство,Казахстан,Дизель,10.8,MT,NaN,Актюбинская область,Актобе,ЗапКазКАМАЗ -Актобе,Юридическое лицо,NaN,1,49950,49950


In [177]:
df['drive_type'].value_counts(dropna=False).head(10)

4WD    14060
2WD     9824
FWD     9035
NaN     4411
RWD     1674
AWD      962
Name: drive_type, dtype: int64

Здесь пропуски — четвёртое самое встречаемое значение.

<br>

Применим функцию.

In [178]:
pt_drive, str_drive = null_analysis(df, 'drive_type')

In [179]:
# вывод строк с пропусками и наблюдениями
print(str_drive)

# раскомментируйте строчку ниже, чтобы увидеть всю сводную таблицу
# pd.set_option('display.max_rows', 100)
pt_drive.sort_values(by=['mode_perc', 'null_perc'], ascending=[False, True])

# раскомментируйте строчку ниже, чтобы снова сделать таблицу компактной
# pd.reset_option('display.max_rows')
pt_drive.sort_values(by=['null_cnt'], ascending=[False])

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 4411 пропусков. В датафрейме: 4411 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме не сходится.
В сводной: 10121 наблюдений. В датафрейме: 10122 наблюдений.




index,brand,model,2WD,4WD,AWD,FWD,RWD,null_cnt,obs_cnt,null_perc,mode_perc
214,Renault,Duster,0,51,0,6,0,650,707,91.9,89.5
219,Renault,Sandero,0,0,0,3,0,438,441,99.3,100.0
210,Ravon,Nexia R3,805,0,0,533,0,361,1699,21.2,60.2
257,Volkswagen,Polo,77,3,0,103,0,361,544,66.4,56.3
217,Renault,Logan,0,0,0,0,0,320,320,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
148,Land Rover,Range Rover Sport,0,0,14,0,0,1,15,6.7,100.0
149,Land Rover,Range Rover Velar,0,0,15,0,0,1,16,6.2,100.0
182,Mercedes-Benz,V-Class,0,0,0,0,4,1,5,20.0,100.0
184,Mercedes-Benz Trucks,Setra,0,0,0,0,0,1,1,100.0,0.0


Сводная таблица открыла некоторые секреты. Она подсвечивает не только пропуски, но и к какому двухколёсному приводу относится та или иная модель.

Проверив эту информацию на сайте автомобильного портала, сделаем списки передних и задних приводов транспорта, которые в таблице значатся как двухколёсные.

In [180]:
# двухколёсные переднеприводные
fwd = ['S3', 'Nexia R3', 'S5', 'R4', 'Polo', '301', 'Partner Panel Van', 'Accent', 'Creta', 'Caddy Maxi Kombi', 'S90',
       'Elantra', 'Sonata', 'Tucson', 'S7', 'ASX', 'Eclipse Cross', 'Outlander', 'Qashqai', 'X-Trail', 'Caddy Kasten']

# двухколёсные заднеприводные
rwd = ['HFF6850G', 'N120', 'NMR', 'N56', 'H350', 'Corolla', 'NPR', 'HD 65', 'T6', 'HD 78', 'Daily', 'HD 35', 'N75', 'Universe', 'Tiguan', 'Actros', 'X200', 'ZK6108HGH',
       'HFF6124G03EV3', 'HFF6127GZ-4', 'BC 095', 'BC 211 MA', 'BS 106 A', 'BS 106 D', '300', '500', 'TQ-1/H-1', 'HD 170', 'HD 79', 'County', 'NQR', 'Stralis']

# полноприводные модели, которые по ошибке считаются 2WD
wd4 = ['Pajero IV', 'Cayenne', 'Macan', 'Panamera']

Произведём замену по условию.

In [181]:
# замена 2WD на FWD
df.loc[(df['model'].isin(fwd)) & (df['drive_type'] == '2WD'), 'drive_type'] = 'FWD'

# замена 2WD на RWD
df.loc[(df['model'].isin(rwd)) & (df['drive_type'] == '2WD'), 'drive_type'] = 'RWD'

# замена 2WD на 4WD
df.loc[(df['model'].isin(wd4)) & (df['drive_type'] == '2WD'), 'drive_type'] = '4WD'

Проверим распределение значений, чтобы убедиться, что двухколёсных 2WD больше нет. 

In [182]:
df['drive_type'].value_counts(dropna=False)

FWD    18188
4WD    14064
NaN     4411
RWD     2341
AWD      962
Name: drive_type, dtype: int64

<br>

Прошерстив сайт агрегатор автомобилей, создадим списки моделей, которым соответствуют их типы приводов.

In [183]:
# передний привод 
fwd_m = ['6', '3', 'Logan', 'Dokker',]

# задний привод
rwd_m = ['HFF6850G', 'Daily', 'Tipper', 'Daily A', 'Daily V', 'HF-D105', 'BWC6665GA5', 'BJ1069VDJEA-F1',
         'BJ3253DMPKB-AD', 'iev', '65115', '6520', '45143', '65116', '5490', '65117', '43253', 'NMR',
         'NPR', 'HD 78', 'HD 35', 'H350', 'HD 65', 'EX8', 'Universe', 'HD 160', '53215', '45142', '54115',
         '53605', '65206', '4308', '66052', '6606', '5299', 'SX3258DR384']

# полный привод
wd4_m = ['CX-5', 'CX-9', '43118', '44108', '58815Z', '65111', '53504', '43502', '4311', '45141', '65225',
         'Koleos', '4320', '5557', '32551', '4320-1951-40', '4320-1951-60']

# заполним модой
mode_m = ['S3', 'S5', 'N120', 'T6', 'N75', 'N56', 'E-PACE', 'I-Pace', 'XJ', 'Discovery Sport', '301',
          'Range Rover', 'Range Rover Evoque', 'Range Rover Sport', 'Range Rover Velar', 'V-Class',
          'Partner Panel Van', 'Nexia R3', 'R4', 'Sandero', 'HILUX DC', 'Polo', 'Teramont', 'Touareg',
          'Caravelle', 'Amarok', 'Transporter Kasten', 'XC60', 'XC90']

Заполним пропуски для моделей из списков.

In [184]:
# заполним пропуски для заднего привода
df.loc[(df['model'].isin(rwd_m)) & (df['drive_type'].isna()), 'drive_type'] = 'RWD'

# заполним пропуски для полного привода
df.loc[(df['model'].isin(wd4_m)) & (df['drive_type'].isna()), 'drive_type'] = '4WD'

# заполним пропуски для переднего привода
df.loc[(df['model'].isin(fwd_m)) & (df['drive_type'].isna()), 'drive_type'] = 'FWD'

# заполним пропуски модой
for model in mode_m:
    df.loc[(df['model'] == model) & (df['drive_type'].isna()), 'drive_type'] = \
    df.loc[(df['model'] == model), 'drive_type'].mode().iloc[0]

<br>

Проверим результат.

In [185]:
# запишем в иные переменные
pt_drive_fill, str_drive_fill = null_analysis(df, 'drive_type')

print(str_drive_fill)

pt_drive_fill.sort_values(by=['brand', 'null_cnt'], ascending=[True, False])

Количество пропущенных значений в сводной таблице и основном датафрейме сходится.
В сводной: 1042 пропусков. В датафрейме: 1042 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме сходится.
В сводной: 2251 наблюдений. В датафрейме: 2251 наблюдений.




index,brand,model,4WD,AWD,FWD,RWD,null_cnt,obs_cnt,null_perc,mode_perc
164,MAN,TG,0,0,0,0,11,11,100.0,0.0
183,Mercedes-Benz Trucks,Actros,0,1,0,2,5,8,62.5,66.7
184,Mercedes-Benz Trucks,Setra,0,0,0,0,1,1,100.0,0.0
214,Renault,Duster,51,0,6,0,650,707,91.9,89.5
215,Renault,Kaptur,0,0,0,0,230,230,100.0,0.0
212,Renault,Arkana,0,0,0,0,55,55,100.0,0.0
218,Renault,Master,0,0,0,0,8,8,100.0,0.0
222,Scania,R-Series,0,0,0,0,7,7,100.0,0.0
221,Scania,P-Series,0,0,0,0,2,2,100.0,0.0
223,Scania,S-Series,0,0,0,0,2,2,100.0,0.0


Остальные пропуски неизвестно как заполнять.\
Оставшиеся модели `Renault` есть как с передним приводов, так и с полным приводом.\
Это касается и других моделей в таблице выше.

<br>

##### `Тип покупателя`

In [186]:
df[df['customer_type'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
2,2,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,5 серия,E класс,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Нур-Султан,Нур-Султан,Автоцентр-Бавария Астана,NaN,NaN,1,45119,45119
3,3,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,5 серия,E класс,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Алматы,Алматы,Автоцентр-Бавария Алматы,NaN,NaN,1,45119,45119
4,4,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,5 серия,E класс,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Алматы,Алматы,Автоцентр-Бавария Алматы,NaN,NaN,1,37560,37560
5,5,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,6 серия,Спортивные автомобили,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Алматы,Алматы,Автоцентр-Бавария Алматы,NaN,NaN,1,53491,53491
6,6,2019-01-01,Автоцентр-Бавария,Внедорожники,BMW,X1,Компактные SUV,2018,Импорт,Германия,Бензин,2.0,AT,4WD,г.Нур-Султан,Нур-Султан,Автоцентр-Бавария Астана,NaN,NaN,2,36397,72795


In [187]:
df['customer_type'].value_counts(dropna=False).head(10)

Физическое лицо     24733
Юридическое лицо     8186
NaN                  7047
Name: customer_type, dtype: int64

Здесь пропуски — третье самое встречаемое значение.\
Можно предположить, что тип покупателя зависит от "размера заказа".\
Рядовой человек приходит в автосалон, чтобы купить себе один автомобиль.\
Но если в "заказе" у нас больше одного-двух автомобилей, то несложно догадаться, что они покупаются для коммерческих целей.\
Следовательно, это юридическое лицо.

<br>

Применим функцию.

In [188]:
# ф-ция пропусков customer_type
def customer(df, column):
    # копия главного датафрейма
    nan_df = df.copy()
    
    # приведение к строковому типу данных
    nan_df[column] = nan_df[column].astype('str')

    # сводная таблица
    nan_df = nan_df.pivot_table(index='quantity', columns=column, aggfunc='size', fill_value=0).reset_index()

    # переименование колонки
    try:
        nan_df = nan_df.rename(columns={'nan': 'null_cnt'})
    except:
        pass
    
    try:
        nan_df = nan_df.rename(columns={'<NA>': 'null_cnt'})
    except:
        pass
    
    # выбор строк, где есть пропуски
    nan_df = nan_df[nan_df['null_cnt'] != 0]
    
    # переместить 'null_cnt' в конец сводной таблицы
    nan_df = nan_df[[col for col in nan_df.columns if col != 'null_cnt'] + ['null_cnt']]

    # переименование индекса
    nan_df = nan_df.rename_axis(columns='index')

    # преобразование числовых колонок в числовой тип
    nan_df.iloc[:, 1:] = nan_df.iloc[:, 1:].astype('Int64')

    # количество наблюдений по модели
    nan_df['obs_cnt'] = nan_df.iloc[:, 1:].sum(axis=1).astype('Int64')

    # процент пропусков по модели
    nan_df['null_perc'] = nan_df['null_cnt'] / nan_df['obs_cnt'] * 100

    # преобразование колонки с процентом
    nan_df['null_perc'] = round(nan_df['null_perc'].astype(float), 1)

    # какой процент занимает модальное значение от всех непропущенных значений
    nan_df['mode_perc'] = nan_df.apply(lambda row: round(row.iloc[1:-3].max() / (row['obs_cnt'] - row['null_cnt']) * 100, 1) if (row['obs_cnt'] - row['null_cnt']) != 0 and row.iloc[1:-3].max() != 0 else 0, axis=1)
    
    # сверка пропусков и наблюдений сводной таблицы с данными датафрейма
    result_str = ''
    if nan_df['null_cnt'].sum() == df[column].isna().sum():
        result_str += f'Количество пропущенных значений в сводной таблице и основном датафрейме сходится.\n'
        result_str += f'В сводной: {nan_df["null_cnt"].sum()} пропусков. В датафрейме: {df[column].isna().sum()} пропусков.\n\n'
    else:
        result_str += 'Количество пропущенных значений в сводной таблице и основном датафрейме не сходится.\n'
        result_str += f'В сводной: {nan_df["null_cnt"].sum()} пропусков. В датафрейме: {df[column].isna().sum()} пропусков.\n\n'

    if nan_df['obs_cnt'].sum() == len(df[df['model'].isin(df[df[column].isna()]['model'].unique())]):
        result_str += 'Количество наблюдений в сводной таблице и основном датафрейме сходится.\n'
        result_str += f'В сводной: {nan_df["obs_cnt"].sum()} наблюдений. В датафрейме: {len(df[df["quantity"].isin(df[df[column].isna()]["quantity"].unique())])} наблюдений.\n\n'
    else: 
        result_str += 'Количество наблюдений в сводной таблице и основном датафрейме не сходится.\n'
        result_str += f'В сводной: {nan_df["obs_cnt"].sum()} наблюдений. В датафрейме: {len(df[df["quantity"].isin(df[df[column].isna()]["quantity"].unique())])} наблюдений.\n\n'
    
    return nan_df, result_str

In [189]:
pt_customer, str_customer = customer(df, 'customer_type')

In [190]:
# вывод строк с пропусками и наблюдениями
print(str_customer)

pt_customer.sort_values(by=['quantity'], ascending=[True]).head()

# раскомментируйте строчку ниже, чтобы увидеть всю сводную таблицу
# pt_customer.sort_values(by=['quantity'], ascending=[True])

Количество пропущенных значений в сводной таблице и основном датафрейме не сходится.
В сводной: 7041 пропусков. В датафрейме: 7047 пропусков.

Количество наблюдений в сводной таблице и основном датафрейме не сходится.
В сводной: 39959 наблюдений. В датафрейме: 39965 наблюдений.




index,quantity,Физическое лицо,Юридическое лицо,null_cnt,obs_cnt,null_perc,mode_perc
0,1,24729,8148,4327,37204,11.6,75.2
1,2,3,11,1003,1017,98.6,78.6
2,3,0,9,443,452,98.0,100.0
3,4,0,0,282,282,100.0,0.0
4,5,0,4,181,185,97.8,100.0


Количество наблюдений и пропусков не сходится на $6$, потому что есть пропуски в `quantity`, их всего $6$.

Сделаем просто:\
Если количество автомобилей в заказе больше одного, то это — юридическое лицо.\
Если всего $1$ автомобиль в заказе, значит — физическое лицо.

In [191]:
# одно ТС в заказе
df.loc[(df['quantity'] == 1) & df['customer_type'].isna(), 'customer_type'] = 'Физическое лицо'

# больше 1 ТС в заказе
df.loc[(df['quantity'] > 1) & df['customer_type'].isna(), 'customer_type'] = 'Юридическое лицо'

<br>

Проверим результат.

In [192]:
df['customer_type'].isna().sum()

6

Всего $6$ пропусков осталось. Это как раз там, где есть пропуски в `quantity`.

Сделаем срез по этим пропускам.

In [193]:
df[df['quantity'].isna()]

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
31874,31874,2019-08-01,БИПЭК АВТО,Внедорожники,Kia,Sportage,Компактные SUV,2019,Локальное производство,Казахстан,Бензин,2.0,AT,4WD,Карагандинская область,Караганда,БИПЭК АВТО Караганда,NaN,NaN,<NA>,20700,20700
31900,31900,2019-08-01,БИПЭК АВТО,Легковые автомобили,Lada,Granta,B класс,2019,Локальное производство,Казахстан,Бензин,1.6,AMT,FWD,г.Алматы,Алматы,БИПЭК АВТО Алматы,NaN,NaN,<NA>,7100,7100
32068,32068,2019-08-01,БИПЭК АВТО,Легковые автомобили,Lada,XRAY,B класс,2018,Локальное производство,Казахстан,Бензин,1.6,MT,FWD,Павлодарская область,Экибастуз,БИПЭК АВТО Экибастуз,NaN,NaN,<NA>,13800,13800
33150,33150,2019-08-01,БИПЭК АВТО,Внедорожники,Skoda,Kodiaq,Полноразмерные SUV,2019,Локальное производство,Казахстан,Бензин,2.0,AMT,4WD,г.Нур-Султан,Нур-Султан,БИПЭК АВТО Астана,NaN,NaN,<NA>,40000,40000
34568,34568,2019-08-01,БИПЭК АВТО,Коммерческие автомобили,UAZ,3909,Развозные автомобили,2018,Локальное производство,Казахстан,Бензин,2.7,MT,4WD,г.Алматы,Алматы,БИПЭК АВТО Алматы,NaN,NaN,<NA>,11200,11200
34593,34593,2019-08-01,БИПЭК АВТО,Внедорожники,UAZ,Patriot,Среднеразмерные SUV,2018,Локальное производство,Казахстан,Бензин,2.7,MT,4WD,Восточно-Казахстанская область,Семей,БИПЭК АВТО Семей,NaN,NaN,<NA>,13800,13800


Думаю, здесь можно присвоить `количеству` цифру $1$, ведь факт продажи был.\
А `тип покупателя` в таком случае будет Физическое лицо.

In [194]:
# "проблемные" индексы
indx = df.loc[df['quantity'].isna()].index

In [195]:
# заменим пропуски
df.loc[df['quantity'].isna(), 'customer_type'] = df.loc[df['quantity'].isna(), 'customer_type'] = 'Физическое лицо'
df.loc[df['quantity'].isna(), 'quantity'] = df.loc[df['quantity'].isna(), 'quantity'] = 1

Проверим, заменились ли пропуски.

In [196]:
df.loc[indx]

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
31874,31874,2019-08-01,БИПЭК АВТО,Внедорожники,Kia,Sportage,Компактные SUV,2019,Локальное производство,Казахстан,Бензин,2.0,AT,4WD,Карагандинская область,Караганда,БИПЭК АВТО Караганда,Физическое лицо,NaN,1,20700,20700
31900,31900,2019-08-01,БИПЭК АВТО,Легковые автомобили,Lada,Granta,B класс,2019,Локальное производство,Казахстан,Бензин,1.6,AMT,FWD,г.Алматы,Алматы,БИПЭК АВТО Алматы,Физическое лицо,NaN,1,7100,7100
32068,32068,2019-08-01,БИПЭК АВТО,Легковые автомобили,Lada,XRAY,B класс,2018,Локальное производство,Казахстан,Бензин,1.6,MT,FWD,Павлодарская область,Экибастуз,БИПЭК АВТО Экибастуз,Физическое лицо,NaN,1,13800,13800
33150,33150,2019-08-01,БИПЭК АВТО,Внедорожники,Skoda,Kodiaq,Полноразмерные SUV,2019,Локальное производство,Казахстан,Бензин,2.0,AMT,4WD,г.Нур-Султан,Нур-Султан,БИПЭК АВТО Астана,Физическое лицо,NaN,1,40000,40000
34568,34568,2019-08-01,БИПЭК АВТО,Коммерческие автомобили,UAZ,3909,Развозные автомобили,2018,Локальное производство,Казахстан,Бензин,2.7,MT,4WD,г.Алматы,Алматы,БИПЭК АВТО Алматы,Физическое лицо,NaN,1,11200,11200
34593,34593,2019-08-01,БИПЭК АВТО,Внедорожники,UAZ,Patriot,Среднеразмерные SUV,2018,Локальное производство,Казахстан,Бензин,2.7,MT,4WD,Восточно-Казахстанская область,Семей,БИПЭК АВТО Семей,Физическое лицо,NaN,1,13800,13800


Да, теперь всё ок.

<br>

##### `Форма оплаты`

In [197]:
df[df['payment_form'].isna()].head()

,id,purchase_date,company,car_type,brand,model,car_classification,manufacturing_year,assembly_type,manufacturing_country,fuel_type,engine_displacement_l,transmission_type,drive_type,region,city,dealer_center_name,customer_type,payment_form,quantity,price_usd,sale_usd
0,0,2019-01-01,Mercur Auto,Легковые автомобили,Audi,A8,F класс,2018,Импорт,Германия,Бензин,3.0,AT,4WD,г.Алматы,Алматы,Mercur Auto Алматы,Юридическое лицо,NaN,1,129935,129935
1,1,2019-01-01,Mercur Auto,Внедорожники,Audi,Q5,Среднеразмерные SUV,2017,Импорт,Германия,Бензин,2.0,AMT,4WD,г.Алматы,Алматы,Mercur Auto Алматы,Физическое лицо,NaN,1,47836,47836
2,2,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,5 серия,E класс,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Нур-Султан,Нур-Султан,Автоцентр-Бавария Астана,Физическое лицо,NaN,1,45119,45119
3,3,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,5 серия,E класс,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Алматы,Алматы,Автоцентр-Бавария Алматы,Физическое лицо,NaN,1,45119,45119
4,4,2019-01-01,Автоцентр-Бавария,Легковые автомобили,BMW,5 серия,E класс,2018,Импорт,Германия,Бензин,2.0,AT,RWD,г.Алматы,Алматы,Автоцентр-Бавария Алматы,Физическое лицо,NaN,1,37560,37560


In [198]:
df['payment_form'].value_counts(dropna=False).head(10)

NaN                25936
Наличный расчёт     7796
В кредит            6234
Name: payment_form, dtype: int64

С пропусками в этой колонке ничего не поделаешь.

<br>

**Пропущенные значения отработаны.**

<br>

### Несуразица в значениях

Пишу из будущего: в процессе анализа были выявлены ещё аномальные и неправдивые значения. Помогало в этом критическое мышление.\
Некоторые грузовики были ошибочно записаны как малотоннажные, а другие ошибочно как крупнотоннажные.\
Ошибки в объёмах двигателей у некоторых авто.\
Ляпы в категоризации класса авто, к примеру, Nissan Terrano записан как полноразмерный внедорожник, хотя это маленький кроссовер (копия Renault Duster).\
Всё это может неприятно повлиять на статистику.\
Исправим это.

In [199]:
# устраняем сказочный объём двигателя
df.loc[(df['model'] == 'R4') & (df['engine_displacement_l'] == 2.5), 'engine_displacement_l'] = 1.5
df.loc[(df['model'] == 'Camry') & (df['engine_displacement_l'] == 4.6), 'engine_displacement_l'] = 2.5
df.loc[(df['model'] == 'Corolla') & (df['engine_displacement_l'].isin([4.0, 2.7])), ['drive_type', 'engine_displacement_l']] = ['FWD', 1.6]
df.loc[(df['model'] == 'Polo') & (df['engine_displacement_l'] == 8.7), 'engine_displacement_l'] = 1.6
df.loc[(df['car_classification'] == 'Большие автобусы') & (df['engine_displacement_l'] == 2.5), 'engine_displacement_l'] = np.NaN

# ошибки в классификации авто
# эти модели имеют малый объём двигателя или у них неверно указан класс авто, поэтому меняем класс
df.loc[df['model'].isin(['Master', 'H350', 'Cargo']), 'car_classification'] = 'Малотоннажные грузовики'
# эти модели имеют большой объём двигателя для малотоннажных, поэтому меняем класс
df.loc[(df['car_classification'] == 'Малотоннажные грузовики') & (df['engine_displacement_l'].isin([5.2, 7.6])), 'car_classification'] = 'Среднетоннажные грузовики'
df.loc[(df['car_classification'] == 'Малотоннажные грузовики') & (df['engine_displacement_l'].isin([8.7, 11.8])), 'car_classification'] = 'Крупнотоннажные грузовики'

<br>

### Несуразица в классификациях

Классификация автомобилей обозначает их физические размеры.\
**Она не отражает их престижность и люксовость.**

Переделаем классификацию автомобилей.
Для большей точности добавим кроссоверам ещё одну категорию, разделив полноразмерные SUV на $2$ части.\
Таким образом, их классификация будет:
* B-SUV — субкомпактные
* C-SUV — компактные
* D-SUV — среднеразмерные
* E-SUV — полноразмерные $1$
* F-SUV — полноразмерные $2$

Также поводом этому решению послужило и то, что некоторые автомобили, как легковые так и внедорожники, были ошибочно записаны не в их классы, например, Terrano в полноразмерные, а Mercedes C-Class в C класс, а не D. И таких немало. Проверять классы мне помогал сайт `auto.ru` + критическое мышление.

Посмотрите, какие классы на данный момент есть в датафрейме.

In [200]:
df['car_classification'].sort_values().unique()

array(['A класс', 'B класс', 'C класс', 'D класс', 'E класс', 'F класс',
       'Pick-ups', 'Большие автобусы', 'Компактвэн', 'Компактные SUV',
       'Крупнотоннажные грузовики', 'Малотоннажные грузовики',
       'Микроавтобусы', 'Полноразмерные SUV', 'Полноразмерный Минивэн',
       'Развозные автомобили', 'Спортивные автомобили',
       'Среднеразмерные SUV', 'Среднетоннажные грузовики',
       'Средние автобусы', 'Субкомпактные SUV'], dtype=object)

Вперёд, исправим.\
В комментариях будут указаны классы, которые будем менять.

<br>

##### `A класс`

In [201]:
# модели А класса
a_class = ['Picanto']

Проверка, не совершил ли я опечатку в списке.

In [202]:
# флаг
all_present = True

for model in a_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        # если есть опечатка в списке, то флаг ставится в False
        all_present = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной a_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной a_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются А классом в датасете.

In [203]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(a_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,A класс,39


Все модели из списка — А класс.

Посмотрим, есть ли модели, которые по ошибке назвали А классом.

In [204]:
df.loc[(df['car_classification'] == 'A класс') & (~df['model'].isin(a_class)), ['brand', 'model']].value_counts().reset_index().rename(columns={0: 'cnt'})

,brand,model,cnt
0,Mercedes-Benz,A-Class,3


Mercedes A-Class сделался A классом.\
Немного каламбура: ведь логично же, раз буква А есть в названии модели, значит и класс авто такой же.\
Но нет, на самом деле это C класс.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [205]:
print(f"Было: {df.loc[df['model'].isin(a_class), 'car_classification'].unique()[0]}")

Было: A класс


In [206]:
# A класс > A
df.loc[df['model'].isin(a_class), 'car_classification'] = 'A'

In [207]:
print(f"Стало: {df.loc[df['model'].isin(a_class), 'car_classification'].unique()[0]}")

Стало: A


<br>

##### `B класс`

In [208]:
# B класс
b_class = ['Aveo', 'Fiesta', 'Accent', 'Rio', 'Granta', 'Kalina', 'Vesta', 'XRAY', 'Cabrio',
           'Hatch', 'Almera', '301', 'Nexia R3', 'R4', 'Logan', 'Sandero', 'Rapid', 'Polo']

Проверка, не совершил ли я опечатку в списке.

In [209]:
# флаг
all_present = True

for model in b_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        # если есть опечатка в списке, то флаг ставится в False
        all_present = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной b_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной b_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются B классом в датасете.

In [210]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(b_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,B класс,7028
1,C класс,20


$20$ автомобилей находятся в C классе, а им следует быть в B классе.

In [211]:
df.loc[(df['model'].isin(b_class)) & ~(df['car_classification'] == 'B класс'), ['brand', 'model']].value_counts().reset_index().rename(columns={0: 'cnt'})

,brand,model,cnt
0,Nissan,Almera,20


Эти $20$ авто — Nissan Almera. Последнее поколение официально относится к B классу.

Посмотрим, есть ли модели, которые по ошибке назвали B классом.

In [212]:
df.loc[(df['car_classification'] == 'B класс') & ~(df['model'].isin(b_class)), ['brand', 'model']].value_counts().reset_index().rename(columns={0: 'cnt'})

,brand,model,cnt


Таких моделей нет.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [213]:
print(f"Было: {df.loc[df['model'].isin(b_class), 'car_classification'].unique()[0]}")

Было: B класс


In [214]:
# B класс > B
df.loc[df['model'].isin(b_class), 'car_classification'] = 'B'

In [215]:
print(f"Стало: {df.loc[df['model'].isin(b_class), 'car_classification'].unique()[0]}")

Стало: B


<br>

##### `C класс`

In [216]:
# C класс
c_class = ['A3', 'Elantra', 'iev', 'Cerato', 'Solano', '3',
           'A-Class', 'CLA-Class', 'Octavia', 'Corolla', 'Jetta']

Проверка, не совершил ли я опечатку в списке.

In [217]:
# флаг
all_present = True

for model in c_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_present = False
        
if all_present == True:
    print(f"Опечаток нет, все модели из переменной c_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной c_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются C классом в датасете.

In [218]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(c_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,C класс,3553
1,A класс,3


$3$ автомобиля находятся в A классе, а им следует быть в C классе.

In [219]:
df.loc[(df['model'].isin(c_class)) & ~(df['car_classification'] == 'C класс'), ['brand', 'model']].value_counts().reset_index().rename(columns={0: 'cnt'})

,brand,model,cnt
0,Mercedes-Benz,A-Class,3


Эти $3$ автомобиля и есть те Mercedes A-Class, которые мы упомянули, когда разбирались с A классом.

Посмотрим, есть ли модели, которые по ошибке назвали C классом.

In [220]:
df.loc[(df['car_classification'] == 'C класс') & ~(df['model'].isin(c_class)), ['brand', 'model']].value_counts().reset_index().rename(columns={0: 'cnt'})

,brand,model,cnt
0,Mercedes-Benz,C-Class,2
1,Jaguar,XE,1


Это Mercedes C-Class и Jaguar XE.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [221]:
print(f"Было: {df.loc[df['model'].isin(c_class), 'car_classification'].unique()[0]}")

Было: C класс


In [222]:
# C класс > C
df.loc[df['model'].isin(c_class), 'car_classification'] = 'C'

In [223]:
print(f"Стало: {df.loc[df['model'].isin(c_class), 'car_classification'].unique()[0]}")

Стало: C


<br>

##### `D класс`

In [224]:
# D класс
d_class = ['A4', '3 серия', 'Mondeo', 'Sonata', 'Q50', 'XE', 'Optima', 'Stinger',
           'Murman', '6', 'C-Class', 'Superb', 'Legacy', 'Outback', 'Camry']

Проверка, не совершил ли я опечатку в списке.

In [225]:
# флаг
all_present = True

for model in d_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_present = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной d_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной d_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются D классом в датасете.

In [226]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(d_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,D класс,5385
1,C класс,3


$3$ автомобиля находятся в C классе, а им следует быть в D классе.

In [227]:
df.loc[(df['model'].isin(d_class)) & ~(df['car_classification'] == 'D класс'), ['brand', 'model']].value_counts()\
                                                                                                  .reset_index()\
                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Mercedes-Benz,C-Class,2
1,Jaguar,XE,1


Эти $3$ автомобиля и есть те Mercedes C-Class и Jaguar XE, которые мы упомянули, когда разбирались с C классом.

Посмотрим, есть ли модели, которые по ошибке назвали D классом.

In [228]:
df.loc[(df['car_classification'] == 'D класс') & ~(df['model'].isin(d_class)), ['brand', 'model']].value_counts()\
                                                                                                  .reset_index()\
                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt


Таких моделей нет.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [229]:
print(f"Было: {df.loc[df['model'].isin(d_class), 'car_classification'].unique()[0]}")

Было: D класс


In [230]:
# D класс > D
df.loc[df['model'].isin(d_class), 'car_classification'] = 'D'

In [231]:
print(f"Стало: {df.loc[df['model'].isin(d_class), 'car_classification'].unique()[0]}")

Стало: D


<br>

##### `E класс`

In [232]:
# E класс
e_class = ['A6', 'A7', '5 серия', 'XF', 'ES', 'E-Class', 'CLS-Class', 'S90', 'V90']

Проверка, не совершил ли я опечатку в списке.

In [233]:
# флаг
all_present = True

for model in e_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_present = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной e_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной e_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются E классом в датасете.

In [234]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(e_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,E класс,244


Все автомобили — E класса.

Посмотрим, есть ли модели, которые по ошибке назвали E классом.

In [235]:
df.loc[(df['car_classification'] == 'E класс') & ~(df['model'].isin(e_class)), ['brand', 'model']].value_counts()\
                                                                                                  .reset_index()\
                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt


Таких моделей нет.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [236]:
print(f"Было: {df.loc[df['model'].isin(e_class), 'car_classification'].unique()[0]}")

Было: E класс


In [237]:
# E класс > E
df.loc[df['model'].isin(e_class), 'car_classification'] = 'E'

In [238]:
print(f"Стало: {df.loc[df['model'].isin(e_class), 'car_classification'].unique()[0]}")

Стало: E


<br>

##### `F класс`

In [239]:
# F класс
f_class = ['A8', '7 серия', 'XJ', 'Quoris', 'LS', 'S-Class', 'Panamera']

Проверка, не совершил ли я опечатку в списке.

In [240]:
# флаг
all_present = True

for model in f_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_present = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной f_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной f_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются F классом в датасете.

In [241]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(f_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,F класс,40


Все автомобили — F класса.

Посмотрим, есть ли модели, которые по ошибке назвали F классом.

In [242]:
df.loc[(df['car_classification'] == 'F класс') & ~(df['model'].isin(f_class)), ['brand', 'model']].value_counts()\
                                                                                                  .reset_index()\
                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt


Таких моделей нет.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [243]:
print(f"Было: {df.loc[df['model'].isin(f_class), 'car_classification'].unique()[0]}")

Было: F класс


In [244]:
# F класс > F
df.loc[df['model'].isin(f_class), 'car_classification'] = 'F'

In [245]:
print(f"Стало: {df.loc[df['model'].isin(f_class), 'car_classification'].unique()[0]}")

Стало: F


<br>

##### `S класс`

S класс — в колонке `car_classification` это `спортивные автомобили`.\
Это действительно спортивные автомобили: купе, кабриолеты.

In [246]:
# S класс
s_class = ['RS5', '6 серия', '8 серия', 'i8', 'Z4', 'Camaro', 'F-Type', '911 Carrera S']

Проверка, не совершил ли я опечатку в списке.

In [247]:
# флаг
all_present = True

for model in s_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_present = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной s_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной s_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются S классом в датасете.

In [248]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(s_class), 'car_classification'].value_counts()\
                                                       .reset_index()\
                                                       .rename(columns={'index': 'class',
                                                                        'car_classification': 'cnt'})

,class,cnt
0,Спортивные автомобили,10


Все автомобили — S класса.

Посмотрим, есть ли модели, которые по ошибке назвали S классом.

In [249]:
df.loc[(df['car_classification'] == 'Спортивные автомобили') & ~(df['model'].isin(s_class)), ['brand', 'model']].value_counts()\
                                                                                                  .reset_index()\
                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt


Таких моделей нет.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [250]:
print(f"Было: {df.loc[df['model'].isin(s_class), 'car_classification'].unique()[0]}")

Было: Спортивные автомобили


In [251]:
# S класс > S
df.loc[df['model'].isin(s_class), 'car_classification'] = 'S'

In [252]:
print(f"Стало: {df.loc[df['model'].isin(s_class), 'car_classification'].unique()[0]}")

Стало: S


<br>

##### `B-SUV класс`

B-SUV класс — это субкомпактные кроссоверы и внедорожники.

In [253]:
# B-SUV класс
b_suv_class = ['Tracker', 'Niva', 'Creta', 'Soul', '4x4', 'X50', '3151', 'S3',
               'Countryman', 'Juke', 'Terrano', 'Duster', 'Kaptur', 'C-HR']

Проверка, не совершил ли я опечатку в списке.

In [254]:
# флаг
all_present = True

for model in b_suv_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_correct = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной b_suv_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной b_suv_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются B-SUV классом в датасете.

In [255]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(b_suv_class), 'car_classification'].value_counts()\
                                                           .reset_index()\
                                                           .rename(columns={'index': 'class',
                                                                            'car_classification': 'cnt'})

,class,cnt
0,Субкомпактные SUV,4888
1,Полноразмерные SUV,327
2,Компактные SUV,85
3,Среднеразмерные SUV,46


Видим полный ассортимент различных типов.\
Посмотрим модели, которые не являются Субкомпактными SUV в датасете.

In [256]:
df.loc[(df['model'].isin(b_suv_class)) & ~(df['car_classification'] == 'Субкомпактные SUV'), ['brand', 'model']].value_counts()\
                                                                                                                .reset_index()\
                                                                                                                .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Nissan,Terrano,327
1,Toyota,C-HR,76
2,UAZ,3151,46
3,Mini,Countryman,9


Nissan Terrano здесь представлен как товарищ Cadillac Escalade, хотя это копия Renault Duster, который совсем небольшой.\
Toyota C-HR и Mini Countryman нужно опустить на класс ниже. Напомню, что классы обозначают размеры авто, а не их статус престижа.\
Уазик — это маленький внедорожник.

Посмотрим, есть ли модели, которые по ошибке посчитали Субкомпактными SUV.

In [257]:
df.loc[(df['car_classification'] == 'Субкомпактные SUV') & ~(df['model'].isin(b_suv_class)), ['brand', 'model']].value_counts()\
                                                                                                                .reset_index()\
                                                                                                                .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Jac,S5,447
1,Lifan,X70,4


Отряд китайцев тут затесался.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [258]:
print(f"Было: {df.loc[df['model'].isin(b_suv_class), 'car_classification'].unique()[0]}")

Было: Субкомпактные SUV


In [259]:
# Субкомпактные SUV > B-SUV
df.loc[df['model'].isin(b_suv_class), 'car_classification'] = 'B-SUV'

In [260]:
print(f"Стало: {df.loc[df['model'].isin(b_suv_class), 'car_classification'].unique()[0]}")

Стало: B-SUV


<br>

##### `C-SUV класс`

C-SUV класс — это компактные кроссоверы и внедорожники.

In [261]:
# C-SUV класс
c_suv_class = ['X1', 'X2', 'Tucson', 'S5', 'E-PACE', 'Sportage', 'Range Rover Evoque', 'UX', 'X60', 'XC40',
               'X70', 'CX-5', 'GLA-Class', 'ASX', 'Eclipse Cross', 'Qashqai', 'Arkana', 'XV', 'RAV4', 'Tiguan']

Проверка, не совершил ли я опечатку в списке.

In [262]:
# флаг
all_present = True

for model in c_suv_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_correct = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной c_suv_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной c_suv_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются C-SUV классом в датасете.

In [263]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(c_suv_class), 'car_classification'].value_counts()\
                                                           .reset_index()\
                                                           .rename(columns={'index': 'class',
                                                                            'car_classification': 'cnt'})

,class,cnt
0,Компактные SUV,5572
1,Субкомпактные SUV,451
2,Среднеразмерные SUV,25


Здесь есть как B-SUV, так и D-SUV.\
Посмотрим модели, которые не являются Компактными SUV в датасете.

In [264]:
df.loc[(df['model'].isin(c_suv_class)) & ~(df['car_classification'] == 'Компактные SUV'), ['brand', 'model']].value_counts()\
                                                                                                             .reset_index()\
                                                                                                             .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Jac,S5,447
1,Lexus,UX,25
2,Lifan,X70,4


2 китайца и 1 японец.

Посмотрим, есть ли модели, которые по ошибке посчитали компактными SUV.

In [265]:
df.loc[(df['car_classification'] == 'Компактные SUV') & ~(df['model'].isin(c_suv_class)), ['brand', 'model']].value_counts()\
                                                                                                             .reset_index()\
                                                                                                             .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Subaru,Forester,291
1,Nissan,X-Trail,280
2,Mitsubishi,Outlander,265
3,Jac,T6,64
4,Chevrolet,Captiva,30
5,Jaguar,I-Pace,4
6,Jac,S7,1
7,Renault,Koleos,1


Здесь все представители D-SUV класса, подсказывает `auto.ru`

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [266]:
print(f"Было: {df.loc[df['model'].isin(c_suv_class), 'car_classification'].unique()[0]}")

Было: Компактные SUV


In [267]:
# Компактные SUV > C-SUV
df.loc[df['model'].isin(c_suv_class), 'car_classification'] = 'C-SUV'

In [268]:
print(f"Стало: {df.loc[df['model'].isin(c_suv_class), 'car_classification'].unique()[0]}")

Стало: C-SUV


<br>

##### `D-SUV класс`

D-SUV класс — это среднеразмерные кроссоверы и внедорожники.

In [269]:
# D-SUV класс
d_suv_class = ['Q5', 'X3', 'X4', 'XT5', 'Captiva', 'Santa Fe', 'QX50', 'S7', 'F-Pace', 'I-Pace', 'Sorento', 
               'Discovery Sport', 'Koleos', 'Kodiaq', 'Forester', 'Range Rover Velar', 'Range Rover Sport',
               'NX', 'GLC-Class', 'GLC Coupe', 'Outlander', 'Pajero Sport', 'X-Trail', 'Macan', 'Patriot', 'XC60']

Проверка, не совершил ли я опечатку в списке.

In [270]:
# флаг
all_present = True

for model in d_suv_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_correct = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной d_suv_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной d_suv_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются D-SUV классом в датасете.

In [271]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(d_suv_class), 'car_classification'].value_counts()\
                                                           .reset_index()\
                                                           .rename(columns={'index': 'class',
                                                                            'car_classification': 'cnt'})

,class,cnt
0,Среднеразмерные SUV,1493
1,Компактные SUV,872
2,Полноразмерные SUV,87


Здесь есть как C-SUV, так и E/F-SUV.

Посмотрим модели, которые не являются среднеразмерными SUV в датасете, а должны являться таковыми.

In [272]:
df.loc[(df['model'].isin(d_suv_class)) & ~(df['car_classification'] == 'Среднеразмерные SUV'), ['brand', 'model']].value_counts()\
                                                                                                                  .reset_index()\
                                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Subaru,Forester,291
1,Nissan,X-Trail,280
2,Mitsubishi,Outlander,265
3,Skoda,Kodiaq,72
4,Chevrolet,Captiva,30
5,Porsche,Macan,15
6,Jaguar,I-Pace,4
7,Jac,S7,1
8,Renault,Koleos,1


Целая вереница несостыковок.

Посмотрим, есть ли модели, которые по ошибке посчитали среднеразмерными SUV.

In [273]:
df.loc[(df['car_classification'] == 'Среднеразмерные SUV') & ~(df['model'].isin(d_suv_class)), ['brand', 'model']].value_counts()\
                                                                                                                  .reset_index()\
                                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Toyota,LC Prado,1711
1,Lexus,RX,442
2,Toyota,HIGHLANDER,197
3,Nissan,Murano,23
4,Chevrolet,Tahoe,20
5,Infiniti,QX60,14
6,Land Rover,Discovery,7
7,Mitsubishi,Pajero IV,7
8,Volkswagen,Touareg,7
9,Lexus,RXH,2


Ошибок много.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [274]:
print(f"Было: {df.loc[df['model'].isin(d_suv_class), 'car_classification'].unique()[0]}")

Было: Среднеразмерные SUV


In [275]:
# Среднеразмерные SUV > D-SUV
df.loc[df['model'].isin(d_suv_class), 'car_classification'] = 'D-SUV'

In [276]:
print(f"Стало: {df.loc[df['model'].isin(d_suv_class), 'car_classification'].unique()[0]}")

Стало: D-SUV


<br>

##### `E-SUV класс`

E-SUV класс — это полноразмерные кроссоверы и внедорожники.

In [277]:
# E-SUV класс
e_suv_class = ['Q7', 'Q8', 'X5', 'X6', 'QX60', 'Discovery', 'GX', 'RX', 'RXH', 'CX-9', 'GLE-Class', 'G-Class',
               'GLE Coupe', 'Pajero IV', 'Murano', 'Cayenne', 'HIGHLANDER', 'LC Prado', 'Touareg', 'Teramont', 'XC90']

Проверка, не совершил ли я опечатку в списке.

In [278]:
# флаг
all_present = True

for model in e_suv_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_correct = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной e_suv_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной e_suv_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются E-SUV классом в датасете.

In [279]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(e_suv_class), 'car_classification'].value_counts()\
                                                           .reset_index()\
                                                           .rename(columns={'index': 'class',
                                                                            'car_classification': 'cnt'})

,class,cnt
0,Среднеразмерные SUV,2410
1,Полноразмерные SUV,234


Видим много D-SUV.

Посмотрим модели, которые не являются полноразмерными E-SUV в датасете, а должны являться таковыми.

In [280]:
df.loc[(df['model'].isin(e_suv_class)) & ~(df['car_classification'] == 'Полноразмерные SUV'), ['brand', 'model']].value_counts()\
                                                                                                                  .reset_index()\
                                                                                                                  .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Toyota,LC Prado,1711
1,Lexus,RX,442
2,Toyota,HIGHLANDER,197
3,Nissan,Murano,23
4,Infiniti,QX60,14
5,Land Rover,Discovery,7
6,Mitsubishi,Pajero IV,7
7,Volkswagen,Touareg,7
8,Lexus,RXH,2


Много чего не сходится.

Посмотрим, есть ли модели, которые по ошибке посчитали E-SUV.

In [281]:
df.loc[(df['car_classification'] == 'Полноразмерные SUV') & ~(df['model'].isin(e_suv_class)), ['brand', 'model']].value_counts()\
                                                                                                                 .reset_index()\
                                                                                                                 .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Toyota,LC200,779
1,Lexus,LX,332
2,Land Rover,Range Rover,85
3,Cadillac,Escalade,70
4,Infiniti,QX80,42
5,BMW,X7,30
6,Mercedes-Benz,GLS-Class,9
7,Chevrolet,Traverse,7


По большому счёту, это не ошибки. Просто они не в своей корзине (F-SUV) полноразмерных внедорожников.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [282]:
print(f"Было: {df.loc[df['model'].isin(e_suv_class), 'car_classification'].unique()[0]}")

Было: Полноразмерные SUV


In [283]:
# Полноразмерные SUV > E-SUV
df.loc[df['model'].isin(e_suv_class), 'car_classification'] = 'E-SUV'

In [284]:
print(f"Стало: {df.loc[df['model'].isin(e_suv_class), 'car_classification'].unique()[0]}")

Стало: E-SUV


<br>

##### `F-SUV класс`

F-SUV класс — это полноразмерные, гигантские кроссоверы и внедорожники.

In [285]:
# F-SUV класс
f_suv_class = ['X7', 'Escalade', 'Tahoe', 'Traverse', 'QX80', 'Range Rover', 'LX', 'GLS-Class', 'LC200']

Проверка, не совершил ли я опечатку в списке.

In [286]:
# флаг
all_present = True

for model in f_suv_class:
    if model not in df['model'].unique():
        print(f"Опечатка. Модель {model} отсутствует в колонке 'model'.")
        all_correct = False

if all_present == True:
    print(f"Опечаток нет, все модели из переменной f_suv_class есть в колонке 'model'.")

Опечаток нет, все модели из переменной f_suv_class есть в колонке 'model'.


Посмотрим, все ли модели из списка являются E-SUV классом в датасете.

In [287]:
# распределение значений классификаций авто, по моделям из списка 
df.loc[df['model'].isin(f_suv_class), 'car_classification'].value_counts()\
                                                           .reset_index()\
                                                           .rename(columns={'index': 'class',
                                                                            'car_classification': 'cnt'})

,class,cnt
0,Полноразмерные SUV,1354
1,Среднеразмерные SUV,20


Среди этих гигантов есть $20$ скромняг.

Посмотрим модели, которые не являются полноразмерными (гигантскими) F-SUV в датасете, а должны являться таковыми.

In [288]:
df.loc[(df['model'].isin(f_suv_class)) & ~(df['car_classification'] == 'Полноразмерные SUV'), ['brand', 'model']].value_counts()\
                                                                                                                 .reset_index()\
                                                                                                                 .rename(columns={0: 'cnt'})

,brand,model,cnt
0,Chevrolet,Tahoe,20


Tahoe — очень большая машина.

Посмотрим, есть ли модели, которые по ошибке посчитали F-SUV.

In [289]:
df.loc[(df['car_classification'] == 'Полноразмерные SUV') & ~(df['model'].isin(f_suv_class)), ['brand', 'model']].value_counts()\
                                                                                                                 .reset_index()\
                                                                                                                 .rename(columns={0: 'cnt'})

,brand,model,cnt


Таких нет.

Переименуем название класса.\
Зафиксируем название до и после переименования.

In [290]:
print(f"Было: {df.loc[df['model'].isin(f_suv_class), 'car_classification'].unique()[0]}")

Было: Полноразмерные SUV


In [291]:
# Полноразмерные SUV > F-SUV
df.loc[df['model'].isin(f_suv_class), 'car_classification'] = 'F-SUV'

In [292]:
print(f"Стало: {df.loc[df['model'].isin(f_suv_class), 'car_classification'].unique()[0]}")

Стало: F-SUV


<br>

Осталась нетронутой только одна модель, которая причислена к Компактным SUV, а она должна быть причислена к пикапам — это Jac T6.

In [293]:
df[df['model'] == 'T6']['car_classification'].unique()[0]

'Компактные SUV'

Поменяем тип.

In [294]:
df.loc[df['model'] == 'T6', 'car_classification'] = 'Pick-ups'

Проверим смену.

In [295]:
df[df['model'] == 'T6']['car_classification'].unique()[0]

'Pick-ups'

Проверим, как теперь выглядят уникальные типы классов авто.

In [296]:
df['car_classification'].sort_values().unique()

array(['A', 'B', 'B-SUV', 'C', 'C-SUV', 'D', 'D-SUV', 'E', 'E-SUV', 'F',
       'F-SUV', 'Pick-ups', 'S', 'Большие автобусы', 'Компактвэн',
       'Крупнотоннажные грузовики', 'Малотоннажные грузовики',
       'Микроавтобусы', 'Полноразмерный Минивэн', 'Развозные автомобили',
       'Среднетоннажные грузовики', 'Средние автобусы'], dtype=object)

Теперь порядок. Теперь можно приступить к анализу данных.

<br>

### Итоги предобработки данных

Посмотрим размер таблицы до и после обработки.

In [297]:
# таблица после предобработки
dry_shape = df.shape

print(f'Размер датафрейма до обработки: {raw_shape[0]} строк, {raw_shape[1]} колонки.')
print(f'Размер датафрейма после обработки: {dry_shape[0]} строк, {dry_shape[1]} колонки.')

Размер датафрейма до обработки: 39966 строк, 25 колонки.
Размер датафрейма после обработки: 39966 строк, 22 колонки.


<br>

**Была проделана следующая работа:**
1. Удалили ненужные колонки.
2. Переименовали колонки и создали новую.
3. Заменили типы данных в колонках.
4. Каждой строке дали по первичному ключу.
5. Устранили неявные дубликаты.
6. Устранили аномальные значения.
7. Разобрались с пропущенными значениями.
8. Устранена несуразица в значениях и классификациях.

Таким образом, датасет готов к анализу.